# <div style="text-align: center"><font color='#dc2624' face='微软雅黑'>FR007 利率掉期定价和曲线拔靴</font></div>

## <font color='#dc2624' face='微软雅黑'>目录</font><a name='toc'></a>
### 1. [**<font color='#dc2624' face='微软雅黑'>简介</font>**](#1)
1. [<font color='#2b4750' face='微软雅黑'>产品介绍</font>](#1.1)
2. [<font color='#2b4750' face='微软雅黑'>定价流程</font>](#1.2)

### 2. [**<font color='#dc2624' face='微软雅黑'>数据处理</font>**](#2)
1. [<font color='#2b4750' face='微软雅黑'>头寸数据</font>](#2.1)
2. [<font color='#2b4750' face='微软雅黑'>市场数据</font>](#2.2)
3. [<font color='#2b4750' face='微软雅黑'>定盘数据</font>](#2.3)

### 3. [**<font color='#dc2624' face='微软雅黑'>日期生成</font>**](#3)
1. [<font color='#2b4750' face='微软雅黑'>基本概念</font>](#3.1)
2. [<font color='#2b4750' face='微软雅黑'>日历创建</font>](#3.2)
3. [<font color='#2b4750' face='微软雅黑'>产品日期</font>](#3.3)
4. [<font color='#2b4750' face='微软雅黑'>指标日期</font>](#3.4)

### 4. [**<font color='#dc2624' face='微软雅黑'>变量计算</font>**](#4)
1. [<font color='#2b4750' face='微软雅黑'>折现因子</font>](#4.1)
2. [<font color='#2b4750' face='微软雅黑'>远期利率</font>](#4.2)

### 5. [**<font color='#dc2624' face='微软雅黑'>曲线构建</font>**](#5)
1. [<font color='#2b4750' face='微软雅黑'>基本概念</font>](#5.1)
1. [<font color='#2b4750' face='微软雅黑'>拔靴方法</font>](#5.2)

### 6. [**<font color='#dc2624' face='微软雅黑'>产品定价</font>**](#6)
1. [<font color='#2b4750' face='微软雅黑'>普通利率掉期 </font>](#6.1)
2. [<font color='#2b4750' face='微软雅黑'>FR007 利率掉期 </font>](#6.2)
---

# <font color='#dc2624' face='微软雅黑'>1. 简介</font><a name='1'></a>
[<font color='black' face='微软雅黑'>回到目录</font>](#toc)
### <font color='#2b4750' face='微软雅黑'>1.1 产品介绍</font><a name='1.1'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#1)

> 定义：<font color='blue'>**七天回购掉期**</font> (FR007 swap) 是指交易双方以一定的名义本金为基础，将该本金产生的一种利率计算的利息收入(支出) 与另一种利率计算的利息收入(支出)。交换的只是不同特征的利息，没有实质本金的互换。

掉期有两端，固定端和浮动端，固定端的利率由一个固定利率决定，而浮动端的利率由若干个七天回购利率 (7D repo rate) 复合计算而得。七天回购掉期的日期表如下图所示。

<img src="FR007.PNG" width=80% height=80%>

把注意力放在浮动端第 <font color='blue'>$n$</font> 期，对应的复合利率 <font color='blue'>$R(T_{n-1},T_{n})$</font> 是由一组七天回购利率 <font color='blue'>$r_{n,1},r_{n,2}, \cdots, r_{n,n_k}$</font> 组成的，展示如下：

<img src="floating rate.PNG" width=70% height=70%>

严格来说，上图展示的七天回购利率首尾连接方式不是很严谨，在此只是为了表达一个“**一组 7D 利率整合成一个复合利率**”的意思。在之后**日期生成**和**利率计算**中，我们会给出上图的严谨版本。

### <font color='#2b4750' face='微软雅黑'>1.2 定价流程</font><a name='1.2'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#1)

为了定价 FR007 掉期，首先需要设定一个<font color='red'>**估值日**</font> `date_eval`，2021 年 2 月 10 日。

In [94]:
import datetime

date_eval = datetime.datetime(2021, 2, 10)

定价产品需要三类数据，<font color='red'>**头寸数据**</font> (instrument data)、<font color='red'>**市场数据**</font>  (market data) 和<font color='red'>**定盘数据**</font> (fixing data)。严格来说定盘数据也是市场数据中的一种，只不过通常市场数据指的是每天的即时数据，而定盘数据指的是历史数据。

In [95]:
from data_loader import load_data

INSTRUMENT, MARKET, FIXING = load_data()

对于 FR007 掉期，市场数据只需要人民币种下的<font color='red'>**零息曲线**</font> (zero curve) 和定盘数据，而利率曲线需要从市场上源报价 (raw market quote) 用拔靴法 (bootstrapping) 得到的。

In [100]:
from curve_construction import bootstrapping

curr, market_index, index = 'CNY', 'FR007', 'FR 7D'
curve = bootstrapping( MARKET, curr, market_index, date_eval )
MARKET[curr][index] = curve

使用类 `IR_CompoundIRS` 来构建 FR007 掉期对象 `cmpirs`，该对象只包含产品的静态特征，比如生效日、到期日、本金、币种等，然后根据这些信息生成产品的对于 FR007 掉期，市场数据只需要人民币种下的<font color='red'>**日期表**</font> (schedule)。

In [102]:
from IR_CompoundIRS_engine import *

In [103]:
deal = INSTRUMENT['IR Compound IRS']['CMPIRS1001']
(date_eff, date_mat, N, curr, discCurve, leg1, f1, dc1, r, leg2, f2, dc2, refCurve, s, payCLD, fixingCLD, BDC) = tuple(deal.values)
(N, r, s) = (float(N), float(r.strip('%')),  float(s.strip('%')))

cmpirs = IR_CompoundIRS( date_eff, date_mat, N, curr, discCurve, 
                         leg1, f1, dc1, r, 
                         leg2, f2, dc2, refCurve, s, 
                         payCLD, fixingCLD, BDC )

对于同样的金融产品，估值方法却可以有很多种。简单掉期产品用折现现金流的方法就可估值，但对于复杂产品可能会用到数值方法如，

- <font color='red'>**数值积分**</font> (numerical integration)
- <font color='red'>**偏微分方程有限差分**</font> (partial differential equation finite difference, PDE FD)
- <font color='red'>**蒙特卡洛**</font> (Monte Carlo, MC)

因此在**静态**对象 `cmpirs` 的基础上，使用类 `IR_CompoundIRS_pricer` 来构建 FR007 掉期定价对象 `cmpirs_engine`。

### <font color='black' face='微软雅黑'>第一笔 FR007 的结果</font>

In [106]:
cmpirs_engine = IR_CompoundIRS_pricer( date_eval, MARKET, cmpirs, FIXING )
cmpirs_engine.NPV()

{'Fixed Leg PV': -4663004.760548228,
 'Floating Leg PV': 5426387.494526284,
 'NPV': 763382.7339780554}

In [107]:
cmpirs_engine.display_fixed_leg()

,Start Date,End Date,Tenor,Payment Date,Discount Factor,Cash Flow,Discounted Cash Flow
0,2021/01/20,2021/04/20,0.2466,2021/04/20,0.9954,"$591,780.82","$589,053.34"
1,2021/04/20,2021/07/20,0.2493,2021/07/20,0.9886,"$598,356.16","$591,563.66"
2,2021/07/20,2021/10/20,0.2521,2021/10/20,0.9819,"$604,931.51","$593,975.86"
3,2021/10/20,2022/01/20,0.2521,2022/01/20,0.9750,"$604,931.51","$589,833.62"
4,2022/01/20,2022/04/20,0.2466,2022/04/20,0.9683,"$591,780.82","$573,009.96"
5,2022/04/20,2022/07/20,0.2493,2022/07/20,0.9614,"$598,356.16","$575,253.26"
6,2022/07/20,2022/10/20,0.2521,2022/10/20,0.9543,"$604,931.51","$577,312.17"
7,2022/10/20,2023/01/20,0.2521,2023/01/20,0.9472,"$604,931.51","$573,002.89"


In [108]:
cmpirs_engine.display_floating_leg()

,Start Date,End Date,Tenor,Compound Rate,Spread,Payment Date,Discount Factor,Cash Flow,Discounted Cash Flow
0,2021/01/20,2021/04/20,0.2466,2.4797%,0.0000%,2021/04/20,0.9954,"$611,443.07","$608,624.96"
1,2021/04/20,2021/07/20,0.2493,2.7376%,0.0000%,2021/07/20,0.9886,"$682,528.24","$674,780.21"
2,2021/07/20,2021/10/20,0.2521,2.7312%,0.0000%,2021/10/20,0.9819,"$688,423.39","$675,955.66"
3,2021/10/20,2022/01/20,0.2521,2.7862%,0.0000%,2022/01/20,0.9750,"$702,280.95","$684,753.42"
4,2022/01/20,2022/04/20,0.2466,2.8319%,0.0000%,2022/04/20,0.9683,"$698,283.35","$676,134.30"
5,2022/04/20,2022/07/20,0.2493,2.8751%,0.0000%,2022/07/20,0.9614,"$716,810.25","$689,133.76"
6,2022/07/20,2022/10/20,0.2521,2.9293%,0.0000%,2022/10/20,0.9543,"$738,352.10","$704,641.18"
7,2022/10/20,2023/01/20,0.2521,2.9837%,0.0000%,2023/01/20,0.9472,"$752,058.02","$712,363.99"


In [109]:
cmpirs_engine.plot().render_notebook()

### <font color='black' face='微软雅黑'>第二笔 FR007 的结果</font>

In [10]:
deal = INSTRUMENT['IR Compound IRS']['CMPIRS1002']
(date_eff, date_mat, N, curr, discCurve, leg1, f1, dc1, r, leg2, f2, dc2, refCurve, s, payCLD, fixingCLD, BDC) = tuple(deal.values)
(N, r, s) = (float(N), float(r.strip('%')),  float(s.strip('%')))

cmpirs = IR_CompoundIRS( date_eff, date_mat, N, curr, discCurve, 
                         leg1, f1, dc1, r, 
                         leg2, f2, dc2, refCurve, s, 
                         payCLD, fixingCLD, BDC )

cmpirs_engine = IR_CompoundIRS_pricer( date_eval, MARKET, cmpirs, FIXING )
cmpirs_engine.NPV()

{'Fixed Leg PV': -7983241.041853355,
 'Floating Leg PV': 12347800.878181417,
 'NPV': 4364559.836328061}

In [11]:
cmpirs_engine.display_fixed_leg()

,Start Date,End Date,Tenor,Payment Date,Discount Factor,Cash Flow,Discounted Cash Flow
0,2021/01/15,2021/04/15,0.2466,2021/04/15,0.9958,"$493,150.68","$491,060.01"
1,2021/04/15,2021/07/15,0.2493,2021/07/15,0.9890,"$498,630.14","$493,153.02"
2,2021/07/15,2021/10/15,0.2521,2021/10/15,0.9823,"$504,109.59","$495,165.33"
3,2021/10/15,2022/01/17,0.2575,2022/01/17,0.9753,"$515,068.49","$502,329.56"
4,2022/01/17,2022/04/15,0.2411,2022/04/15,0.9687,"$482,191.78","$467,078.47"
5,2022/04/15,2022/07/15,0.2493,2022/07/15,0.9618,"$498,630.14","$479,567.54"
6,2022/07/15,2022/10/17,0.2575,2022/10/17,0.9546,"$515,068.49","$491,671.01"
7,2022/10/17,2023/01/16,0.2493,2023/01/16,0.9475,"$498,630.14","$472,467.39"
8,2023/01/16,2023/04/17,0.2493,2023/04/17,0.9404,"$498,630.14","$468,936.43"
9,2023/04/17,2023/07/17,0.2493,2023/07/17,0.9333,"$498,630.14","$465,386.02"


In [12]:
cmpirs_engine.display_floating_leg()

,Start Date,End Date,Tenor,Compound Rate,Spread,Payment Date,Discount Factor,Cash Flow,Discounted Cash Flow
0,2021/01/15,2021/04/15,0.2466,2.4548%,0.0010%,2021/04/15,0.9958,"$629,943.99","$627,273.39"
1,2021/04/15,2021/07/15,0.2493,2.7354%,0.0010%,2021/07/15,0.9890,"$706,912.83","$699,147.87"
2,2021/07/15,2021/10/15,0.2521,2.7299%,0.0010%,2021/10/15,0.9823,"$713,279.40","$700,623.91"
3,2021/10/15,2022/01/17,0.2575,2.7830%,0.0010%,2022/01/17,0.9753,"$742,477.62","$724,114.29"
4,2022/01/17,2022/04/15,0.2411,2.8306%,0.0010%,2022/04/15,0.9687,"$706,556.32","$684,410.76"
5,2022/04/15,2022/07/15,0.2493,2.8722%,0.0010%,2022/07/15,0.9618,"$741,005.18","$712,676.61"
6,2022/07/15,2022/10/17,0.2575,2.9272%,0.0010%,2022/10/17,0.9546,"$779,614.82","$744,200.06"
7,2022/10/17,2023/01/16,0.2493,2.9815%,0.0010%,2023/01/16,0.9475,"$768,262.09","$727,951.96"
8,2023/01/16,2023/04/17,0.2493,3.0202%,0.0010%,2023/04/17,0.9404,"$777,902.98","$731,578.42"
9,2023/04/17,2023/07/17,0.2493,3.0600%,0.0010%,2023/07/17,0.9333,"$787,827.16","$735,302.02"


In [13]:
cmpirs_engine.plot().render_notebook()

Start Date,End Date,Tenor,Payment Date,Discount Factor,Cash Flow,Discounted Cash Flow
2021/01/15,2021/04/15,0.2465753424657534,2021/04/15,0.9957605746541153,493150.6849315068,491060.00941846776
2021/04/15,2021/07/15,0.2493150684931507,2021/07/15,0.989015672475038,498630.1369863014,493153.02024782717
2021/07/15,2021/10/15,0.25205479452054796,2021/10/15,0.9822573103141062,504109.5890410959,495165.32903505635
2021/10/15,2022/01/17,0.25753424657534246,2022/01/17,0.975267491932712,515068.4931506849,502329.5574886297
2022/01/17,2022/04/15,0.2410958904109589,2022/04/15,0.9686570529059799,482191.7808219178,467078.4693464451
2022/04/15,2022/07/15,0.2493150684931507,2022/07/15,0.9617700705996821,498630.1369863014,479567.54205244425
2022/07/15,2022/10/17,0.25753424657534246,2022/10/17,0.9545740357846099,515068.4931506849,491671.01021234697
2022/10/17,2023/01/16,0.2493150684931507,2023/01/16,0.9475307499259165,498630.1369863014,472467.3876342926
2023/01/16,2023/04/17,0.2493150684931507,2023/04/17,0.9404494339475966,498630.1369863014,468936.4300779797
2023/04/17,2023/07/17,0.2493150684931507,2023/07/17,0.9333291067798647,498630.1369863014,465386.02036694624


整套流程展示完毕，接下来我们各点击破每个环节的细节，更重要的是分享笔者处理此类问题的思路：

- 第二节会讲解<font color='red'>**数据处理**</font>，包括如何从**中国外汇交易中心**收集 FR007 的市场数据和定盘数据，如何从 excel 或 csv 中读取数据，如何用 `cufflinks` 来可视化数据。
- 第三节会介绍<font color='red'>**日期生成**</font>，FR007 掉期的产品日期表和指标日期表是如何生成的。
- 第四节会介绍<font color='red'>**变量计算**</font>，如何计算或插值折现因子和远期利率。
- 第五节会讲解<font color='red'>**曲线构建**</font>，如何从市场报价通过拔靴法得到零息曲线。
- 第六节会讲解<font color='red'>**产品定价**</font>，使用面向对象 (object-orient) 方法构建 FR007 掉期对象和定价对象（分别是 IRS 对象和定价对象的子类）。


### <font color='black'>`整套代码架构如下：`</font>

    |--- IR_CMPIRS_engine.py
    |    |--- class IR_CMPIR( IR_InterestRateSwap )
    |    |--- class IR_CMPIR_pricer( IR_InterestRateSwap_pricer )
    |
    |--- IR_InterestRateSwap_engine.py
    |    |--- class IR_InterestRateSwap()
    |    |--- class IR_InterestRateSwap_pricer()
    |
    |--- data_loader.py
    |    |--- load_data()
    |    |    |--- read_instrument()
    |    |    |--- read_market()
    |    |    |--- read_fixing()
    |
    |--- schedule.py
    |--- market_variable.py
    |--- curve_construction.py
    |
    |--- utils.py
    |--- ql_utils.py
    |--- date_utils.py
    |--- daycount_utils.py
    |--- calendar_utils.py
    |--- convention_utils.py
    |--- formatter.py

# <font color='#dc2624' face='微软雅黑'>2. 数据处理</font><a name='2'></a>
[<font color='black' face='微软雅黑'>回到目录</font>](#toc)

### <font color='red' face='微软雅黑'>本节所有代码都来自 `data_load.py` 文件，专门用来读取数据。</font>
首先引入工具包。

In [14]:
import os
import pandas as pd

### <font color='#2b4750' face='微软雅黑'>2.1 头寸数据</font><a name='2.1'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#2)

In [15]:
def read_instrument( path, file_type ):    
    obj_dict = {}
    file_type = file_type.lower()
        
    for r, d, f in os.walk(path):
        for file in f:
            if file_type in ['xls','xlsx']:
                if file_type in file:
                    df = pd.read_excel( os.path.join(r, file), sheet_name=None, index_col=0 )
            else:
                if file_type in file:
                    df = pd.read_csv( os.path.join(r, file), index_col=0 )
            
            key = file.split('.')[0]
            obj_dict[key] = df
            
    return obj_dict

所有头寸数据存储在文件夹 INSTRUMENT 下面，格式为 **csv** 或者 **excel**，验证上面函数 `read_instrument()` 的结果。

In [110]:
directory = os.getcwd() 
INSTRUMENT = read_instrument( directory + '\\INSTRUMENT', 'csv' )
INSTRUMENT['IR Compound IRS']

,CMPIRS1001,CMPIRS1002
ID,,
Effective Date,20-Jan-21,15-Apr-15
Maturity Date,20-Jan-23,15-Apr-25
Notional,100000000,100000000
Currency,CNY,CNY
DiscountCurve,FR007,FR007
Fixed Leg,Pay,Pay
Fixed Frequency,Quarter,Quarter
Fixed Daycount,act/365,act/365
Rate,2.4,2


由上面结果可知，函数 `read_instrument()` 成功读取了两笔 FR007 掉期的信息。

### <font color='#2b4750' face='微软雅黑'>2.2 市场数据</font><a name='2.2'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#2)

FR007 的市场报价数据可从**中国外汇交易中心**网页中 http://www.chinamoney.com.cn/chinese/index.html 收集到，按照以下步骤
    
    基准 --> 利率互换曲线 --> FR007 --> 历史数据

再在

- **日期**中选定合适的起始日和终止日，比如 `2021-01-11 到 2021-2-10`
- **曲线名称**下拉菜单中选定`FR007利率互换收盘曲线`，其他还有选项`FR007利率互换定盘曲线`和`FR007利率互换行情曲线`
- **时刻**下拉菜单中选定`16:30`，其他还有选项`定盘对应曲线的 12:00`和`行情曲线对应 9:30 到 16:00`
- **价格类型**下拉菜单中选定`均值`，其他还有选项`报买`和`报卖`

最终可到达下图的界面，

<img src="FR007 Quote Screen.png" width=100% height=100%>

点击**导入 excel** 按钮将历史定盘数据存成 excel 格式。

In [17]:
def read_market( path, file_type, col ):    
    obj_dict = {}
    file_type = file_type.lower()
        
    for r, d, f in os.walk(path):
        for file in f:
            if file_type in ['xls','xlsx']:
                if file_type in file:
                    df = pd.read_excel( os.path.join(r, file), sheet_name=None, parse_dates=[col], index_col=col )
                    for k, v in df.items():
                        v = v.dropna(axis=0)
                        v.index = pd.to_datetime(v.index)
                        df[k] = v
            else:
                if file_type in file:
                    df = pd.read_csv( os.path.join(r, file), parse_dates=[col], index_col=col )
                    df = df.dropna(axis=0)
                    df.index = pd.to_datetime(df.index)
            
            key = file.split('.')[0]
            obj_dict[key] = df
            
    return obj_dict

所有市场数据存储在文件夹 MARKET 下面，格式为 **csv** 或者 **excel**，验证上面函数 `read_market()` 的结果。

In [111]:
MARKET = read_market( directory + '\\MARKET', 'xlsx', '日期' )
MARKET['CNY']['FR007']

,曲线名称,时刻,价格类型,1M,3M,6M,9M,1Y,2Y,3Y,4Y,5Y,7Y,10Y
日期,,,,,,,,,,,,,,
2021-02-10,FR007利率互换收盘曲线,16:30,均值,2.3087,2.5909,2.6475,2.6763,2.7100,2.8139,2.9068,2.9901,3.0807,3.1849,3.3374
2021-02-09,FR007利率互换收盘曲线,16:30,均值,2.6007,2.6025,2.6348,2.6613,2.6881,2.7910,2.8829,2.9653,3.0579,3.1775,3.3200
2021-02-08,FR007利率互换收盘曲线,16:30,均值,2.4899,2.6196,2.6628,2.6797,2.7062,2.8054,2.8982,2.9827,3.0756,3.1881,3.3519
2021-02-07,FR007利率互换收盘曲线,16:30,均值,2.4439,2.6100,2.6375,2.6538,2.6825,2.7888,2.8738,2.9588,3.0425,3.1738,3.3088
2021-02-05,FR007利率互换收盘曲线,16:30,均值,2.4174,2.6183,2.6550,2.6725,2.6875,2.7901,2.8838,2.9642,3.0475,3.1742,3.3232
2021-02-04,FR007利率互换收盘曲线,16:30,均值,2.4034,2.6200,2.6522,2.6705,2.6852,2.7847,2.8747,2.9561,3.0370,3.1655,3.3105
2021-02-03,FR007利率互换收盘曲线,16:30,均值,2.2613,2.5938,2.6249,2.6437,2.6606,2.7599,2.8495,2.9305,3.0111,3.1316,3.2736
2021-02-02,FR007利率互换收盘曲线,16:30,均值,2.3533,2.5438,2.5675,2.5750,2.5925,2.6935,2.7800,2.8619,2.9442,3.0650,3.2075
2021-02-01,FR007利率互换收盘曲线,16:30,均值,3.0339,2.6283,2.5863,2.5850,2.5913,2.6834,2.7662,2.8457,2.9312,3.0507,3.1900


可视化数据可使用 `cufflinks` 工具包，直接可在 `dataframe` 操作，绘图函数 `df.iplot()` 也非常标准化，便于快速画出美观的图。

In [19]:
import cufflinks as cf
cf.set_config_file(offline=True)

为了在画图时便于操作和美观呈现，

- 删除画图时不需要的列标签信息，`'曲线名称'、'时刻'、'价格类型'`
- 将日期转换成 `YYYY-MM-DD` 的形式

In [112]:
df = MARKET['CNY']['FR007'].drop(columns=['曲线名称', '时刻', '价格类型'])
df.index = df.index.strftime('%Y-%m-%d')
df.head()

,1M,3M,6M,9M,1Y,2Y,3Y,4Y,5Y,7Y,10Y
日期,,,,,,,,,,,
2021-02-10,2.3087,2.5909,2.6475,2.6763,2.7100,2.8139,2.9068,2.9901,3.0807,3.1849,3.3374
2021-02-09,2.6007,2.6025,2.6348,2.6613,2.6881,2.7910,2.8829,2.9653,3.0579,3.1775,3.3200
2021-02-08,2.4899,2.6196,2.6628,2.6797,2.7062,2.8054,2.8982,2.9827,3.0756,3.1881,3.3519
2021-02-07,2.4439,2.6100,2.6375,2.6538,2.6825,2.7888,2.8738,2.9588,3.0425,3.1738,3.3088
2021-02-05,2.4174,2.6183,2.6550,2.6725,2.6875,2.7901,2.8838,2.9642,3.0475,3.1742,3.3232


该数据包含 2021 年 1 月 11 日到 2021 年 2 月 10 日（我们选定的估值日）的 FR007 在 1M 到 10Y 的市场报价。我们可以将该数据以两种形式展示：

1. 横轴为日期，纵轴是报价，标注是期限，展示每个期限的报价随着时间的走势。
2. 横轴为期限，纵轴是报价，标注是日期，展示每个日期的报价曲线。

In [21]:
df.reset_index().iplot( x='日期',
                        mode='lines+markers', size=5,  xTitle='日期', yTitle='报价 (%)', 
                        title='人民币 FR007 市场报价 (2021-01-11 至 2021-02-10)', theme='solar')

In [22]:
df.T.assign(期限=df.columns).iplot( x='期限', mode='lines+markers', size=5,  xTitle='期限', yTitle='报价 (%)', 
                                    title='人民币 FR007 市场报价 (2021-01-11 至 2021-02-10)', theme='solar')

### <font color='#2b4750' face='微软雅黑'>2.3 定盘数据</font><a name='2.3'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#2)

FR007 的定盘数据可从**中国外汇交易中心**网页中 http://www.chinamoney.com.cn/chinese/index.html 收集到，按照以下步骤
    
    基准 --> 本币市场 --> 回购定盘数据 --> 历史数据

可到达下图的界面，

<img src="FR007 Fixing Screen.png" width=100% height=100%>

点击**导入 excel** 按钮将历史定盘数据存成 excel 格式，并将每个曲线分别整理到 excel 的一个工作页，页名可将原有名称

    FR001(%), FR007(%), FR014(%), FDR001(%), FDR007(%), FDR014(%)

改成以下名称
    
    FR 1D, FR 7D, FR 14D, FDR 1D, FDR 7D, FDR 14D
    
以上这种**`曲线缩写 期限`**的格式便于我们计算日期，在后面会详细介绍。

In [23]:
def read_fixing( path, file_type, col ):    
    return read_market( path, file_type, col )

所有市场数据存储在文件夹 FIXING 下面，格式为 **csv** 或者 **excel**，验证上面函数 `read_fixing()` 的结果。

In [113]:
FIXING = read_fixing( directory + '\\FIXING', 'xlsx', 'Date' )
FIXING['CNY']

{'FR 1D':             Rate
 Date            
 2021-05-17  2.17
 2021-05-14  1.82
 2021-05-13  1.97
 2021-05-12  1.96
 2021-05-11  1.95
 ...          ...
 2020-04-08  0.90
 2020-04-07  0.85
 2020-04-03  1.03
 2020-04-02  1.30
 2020-04-01  1.42
 
 [281 rows x 1 columns],
 'FR 7D':             Rate
 Date            
 2021-05-17  2.18
 2021-05-14  2.05
 2021-05-13  2.07
 2021-05-12  2.00
 2021-05-11  2.00
 ...          ...
 2020-04-08  1.40
 2020-04-07  1.40
 2020-04-03  1.50
 2020-04-02  1.70
 2020-04-01  1.94
 
 [281 rows x 1 columns],
 'FR 14D':             Rate
 Date            
 2021-05-17  2.15
 2021-05-14  2.10
 2021-05-13  2.05
 2021-05-12  2.10
 2021-05-11  2.00
 ...          ...
 2020-04-08  1.30
 2020-04-07  1.30
 2020-04-03  1.50
 2020-04-02  1.60
 2020-04-01  1.80
 
 [281 rows x 1 columns],
 'FDR 1D':               Rate
 Date              
 2021-05-17  2.1500
 2021-05-14  1.8018
 2021-05-13  1.9600
 2021-05-12  1.9500
 2021-05-11  1.9302
 ...            ...
 2020-04-08  0.8900

In [114]:
df = pd.concat( [fixing for fixing in FIXING['CNY'].values()], axis=1 )
df.columns = FIXING['CNY'].keys()
df

,FR 1D,FR 7D,FR 14D,FDR 1D,FDR 7D,FDR 14D
Date,,,,,,
2021-05-17,2.17,2.18,2.15,2.1500,2.18,2.10
2021-05-14,1.82,2.05,2.10,1.8018,2.00,2.05
2021-05-13,1.97,2.07,2.05,1.9600,2.07,2.04
2021-05-12,1.96,2.00,2.10,1.9500,2.01,2.00
2021-05-11,1.95,2.00,2.00,1.9302,2.00,1.97
...,...,...,...,...,...,...
2020-04-08,0.90,1.40,1.30,0.8900,1.20,1.12
2020-04-07,0.85,1.40,1.30,0.8100,1.30,1.15
2020-04-03,1.03,1.50,1.50,1.0000,1.40,1.41


In [115]:
df.reset_index().iplot( x='Date',
                        mode='lines+markers', size=5,  xTitle='日期', yTitle='定盘利率 (%)', 
                        title='人民币市场六大利率定盘 (2020-04-01 至 2021-05-17)', theme='solar')

# <font color='#dc2624' face='微软雅黑'>3. 日期生成</font><a name='3'></a>
[<font color='black' face='微软雅黑'>回到目录</font>](#toc)

### <font color='#2b4750' face='微软雅黑'>3.1 基本概念</font><a name='3.1'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#3)

### <font color='red' face='微软雅黑'> 本节所有代码都来自 `date_utils.py` 文件，专门用来处理日期有关的问题。</font>

日期通常用字符串表示，但是根据不同用户习惯，日期字符串都很多种表达方式，如：

- `2021-02-10`：属于 `'dddd-dd-dd'`，年月日都是数字，用 `-` 连接，年有 4 位，月和日都是两位，顺序是`年-月-日`。
- `02/10/2021`：属于 `'dd/dd/dddd'`，年月日都是数字，用 `/` 隔开，年有 4 位，月和日都是两位，顺序是`月/日/年`。
- `10.02.2021`：属于 `'dd.dd.dddd'`，年月日都是数字，用 `.` 隔开，年有 4 位，月和日都是两位，顺序是`日.月.年`。
- `10-Feb-2021`：属于 `'dd-LLL-dddd'`，年和日是数字，月是英文名前三个字母，用 `-` 连接，顺序是`日-月-年`。

等等。

当时月份数和天数为单数时，日期的多种表达方式会引起两个问题：

1. 可以省去第一位的零，比如 `02/10/2021` 和 `2/10/2021` 的月份数分别用 `02` 和 `2` 来表示。
2. 月日混淆：`2021-07-08` 到底是 2021 年 7 月 8 日还是 8 月 7 日？`08.07.2021` 呢?

第一个问题可以下面的 `DATE_FORMAT_STR` 映射解决，而第二个问题需要强行记下来，`2021-07-08` 和 `08.07.2021` 都是 2021 年 7 月 8 日，有的月份放在日期前面，有的日期放在月份前面，因此建议写一个无歧义的格式，比如 `10-Feb-2021`，或者使用 ISO 格式 `2021-02-10`。

In [27]:
DATE_FORMAT_STR = dict({
    'dd-LLL-dddd dd:dd:dd': '%d-%b-%Y %H:%M:%S',
    'dd-LLL-dddd': '%d-%b-%Y',
    'd-LLL-dddd': '%d-%b-%Y',

    'dd/dd/dd': '%m/%d/%y',
    'd/dd/dd': '%m/%d/%y',
    'dd/d/dd': '%m/%d/%y',
    'd/d/dd': '%m/%d/%y',
 
    'dd/dd/dddd': '%m/%d/%Y',
    'dd/d/dddd': '%m/%d/%Y',
    'd/dd/dddd': '%m/%d/%Y',
    'd/d/dddd': '%m/%d/%Y',
 
    'dddd-LLL-dd': '%Y-%b-%d',
    'dddd-LLL-d': '%Y-%b-%d',
 
    'dd-LLL-dd': '%d-%b-%y',
    'd-LLL-dd': '%d-%b-%y',
    
    'dddd-dd-dd': '%Y-%m-%d',
    'dddd-dd-d': '%Y-%m-%d',
    'dddd-d-dd': '%Y-%m-%d',
    'dddd-d-d': '%Y-%m-%d',
 
    'dd.dd.dddd': '%d.%m.%Y',
    'd.dd.dddd': '%d.%m.%Y',
    'dd.d.dddd': '%d.%m.%Y',
    'd.d.dddd': '%d.%m.%Y',
 
    "d LLL dddd": '%d %b %Y',
    "dd LLL dddd": '%d %b %Y',
    "dLLLdddd": '%d%b%Y',
    "ddLLLdddd": '%d%b%Y',
    "dddddddd": '%Y%m%d'
})

def get_datetime_format(dtstr):
    """ Try to recognize date representation format from the date string
    """
    dtstr = dtstr.lower()
    translator = str.maketrans("0123456789abcdefghijklmnopqrstuvwxyz", 
                               "ddddddddddLLLLLLLLLLLLLLLLLLLLLLLLLL")
    fmtstring = dtstr.translate(translator)
    if fmtstring in DATE_FORMAT_STR:
        return DATE_FORMAT_STR[fmtstring]
    else:
        raise ValueError("Incorrect date format string: %s" % fmtstring)

验证 2021 年 7 月 8 日的四种日期格式都是 `%Y-%m-%d`。

In [28]:
print( get_datetime_format('2021-07-08') )
print( get_datetime_format('2021-7-08') )
print( get_datetime_format('2021-07-8') )
print( get_datetime_format('2021-7-8') )

%Y-%m-%d
%Y-%m-%d
%Y-%m-%d
%Y-%m-%d


日期是字符串格式，但是在日期上做操作时需要将其转换成 Python 自带的 `datetime.date` 格式，或者 `QuantLib` 中的 `QuantLib.Date` 格式。因此编写下面四个函数为了便于不同格式的日期之间的相互转换：

- `asdatetime()`：将任意格式的日期 `dt` 转成 `datetime.date` 格式
- `asdatetimes()`：将任意格式的日期**列表** `dt_arr` 转成 `datetime.date` 格式
- `asqldate()`：将任意格式的日期转成 `QuantLib.Date` 格式
- `asqldates()`：将任意格式的日期**列表** `dt_arr` 转成 `QuantLib.Date` 格式

In [29]:
import QuantLib as ql
import datetime


def asdatetime(dt):
    """ Extract datetime from several possible representations
    """
    if isinstance(dt, str):
        return datetime.datetime.strptime(dt, get_datetime_format(dt))
    elif isinstance(dt, datetime.date):
        return datetime.datetime(dt.year, dt.month, dt.day)
    elif isinstance(dt, ql.Date):
        return datetime.datetime(dt.year(), dt.month(), dt.dayOfMonth())
    else:
        raise ValueError("Cannot extract date from: %s" % repr(dt))

def asdatetimes(dt_arr):
    return [asdatetime(dt) for dt in dt_arr]
              
def asqldate(dt):
    """ Extract QuantLib from several possible representations
    """
    if isinstance(dt, str):
        d = datetime.datetime.strptime(dt, get_datetime_format(dt))
        return ql.Date(d.day, d.month, d.year)
    elif isinstance(dt, datetime.date):
        return ql.Date(dt.day, dt.month, dt.year)
    elif isinstance(dt, ql.Date):
        return dt
    else:
        raise ValueError("Cannot extract date from: %s" % repr(dt))
    
def asqldates(dt_arr):
    return [asqldate(dt) for dt in dt_arr]

检验函数 `asdatetime()` 的结果。

In [116]:
dt = datetime.date(2021, 2, 10)
print( asdatetime(dt) )
dt = '2021-02-10'
print( asdatetime(dt) )
dt = ql.Date(10, 2, 2021)
print( asdatetime(dt) )

2021-02-10 00:00:00
2021-02-10 00:00:00
2021-02-10 00:00:00


检验函数 `asdatetimes()` 的结果。

In [117]:
dt = [ datetime.date(2021, 2, 10), datetime.date(2021, 7, 12), datetime.date(2021, 10, 31) ]
print( asdatetimes(dt) )
dt = ['2021-02-10', '12-Jul-2021', '31.10.2021']
print( asdatetimes(dt) )
dt = [ ql.Date(10, 2, 2021), ql.Date(12, 7, 2021), ql.Date(31, 10, 2021) ]
print( asdatetimes(dt) )

[datetime.datetime(2021, 2, 10, 0, 0), datetime.datetime(2021, 7, 12, 0, 0), datetime.datetime(2021, 10, 31, 0, 0)]
[datetime.datetime(2021, 2, 10, 0, 0), datetime.datetime(2021, 7, 12, 0, 0), datetime.datetime(2021, 10, 31, 0, 0)]
[datetime.datetime(2021, 2, 10, 0, 0), datetime.datetime(2021, 7, 12, 0, 0), datetime.datetime(2021, 10, 31, 0, 0)]


检验函数 `asqldate()` 的结果。

In [118]:
dt = datetime.date(2021, 2, 10)
print( asqldate(dt) )
dt = '2021-02-10'
print( asqldate(dt) )
dt = ql.Date(10, 2, 2021)
print( asqldate(dt) )

February 10th, 2021
February 10th, 2021
February 10th, 2021


检验函数 `asqldates()` 的结果。

In [119]:
dt = [ datetime.date(2021, 2, 10), datetime.date(2021, 7, 12), datetime.date(2021, 10, 31) ]
print( asqldates(dt) )
dt = ['2021-02-10', '12-Jul-2021', '31.10.2021']
print( asqldates(dt) )
dt = [ ql.Date(10, 2, 2021), ql.Date(12, 7, 2021), ql.Date(31, 10, 2021) ]
print( asqldates(dt) )

[Date(10,2,2021), Date(12,7,2021), Date(31,10,2021)]
[Date(10,2,2021), Date(12,7,2021), Date(31,10,2021)]
[Date(10,2,2021), Date(12,7,2021), Date(31,10,2021)]


### <font color='#2b4750' face='微软雅黑'>3.2 日历创建</font><a name='3.2'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#3)

### <font color='red' face='微软雅黑'> 本节所有代码都来自 `calendar_utils.py` 文件，专门用来处理日历有关的问题。</font>

工作日 (working days) 就是工作的日子，在金融工程背景下关心的是一个国家或地区的金融中心 (financial center) 的工作日。与之对应的是非工作日 (non-working days)，包含

- **周末**(weekend): 大多是周六和周日，对于中东的金融市场是周五和周日
- **节假日** (holidays)：每个国家或地区都不同

日历 (calendar) 记录中每个金融中心包含的非工作日 (周末和节假日)。在 `QuantLib` 中

    Argentina, Australia, Austria, BespokeCalendar, Botswana, Brazil, Canada, China, CzechRepublic, Denmark, Finland,
    France, Germany, HongKong, Hungary, Iceland, India, Indonesia, Israel, Italy, Japan, JointCalendar, Mexico,
    NewZealand, Norway, NullCalendar, Poland, Romania, Russia, SaudiArabia, Singapore, Slovakia, SouthAfrica,
    SouthKorea, Sweden, Switzerland, Taiwan, TARGET, Thailand, Turkey, Ukraine, UnitedKingdom, UnitedStates, 
    WeekendsOnly
    
创建美国的日历使用 `ql.UnitedStates()`。

In [120]:
cal = ql.UnitedStates()
cal

<QuantLib.QuantLib.UnitedStates; proxy of <Swig Object of type 'QuantLib::UnitedStates *' at 0x0000029E97EEE4B0> >

检查 2021 年 5 月 30 日 (星期日)、2021 年 6 月 1 日 (星期二) 和 2021 年 12 月 25 日 (圣诞节) 是否是节假日。

In [35]:
print( cal.isHoliday( ql.Date(30, 5, 2021) ) )
print( cal.isHoliday( ql.Date(1, 6, 2021) ) )
print( cal.isHoliday( ql.Date(25, 12, 2021) ) )

True
False
True


业界通常用一个 4 个字母的代码来代表每个金融中心对应的财务日历，最常见的是

- 纽约的 USNY
- 欧元区的 EUTA
- 伦敦的 GBLO
- 东京的 JPTY
- 北京的 CNBJ
- 香港 HKHK，
- 悉尼的 AUSY，
- 多伦多的 CATR
- 新加坡的 SGSG

前两个字母代表国家或地区，后两个字母代表城市 (金融中心)。

不同系统也可能用不同代号，比如只用前两个字母 US 代表美国市场；还有一种可能，用货币 USD 代表美国市场。这样 USNY, US 和 USD 可能都需要美国日历。

这样可以创建一个**日历映射** `CALENDAR_MAP`，将 USNY, US, USD 三种代号方式都映射到 `ql.UnitedStates()`。

---

对于中国大陆市场，每年春节、五一、十一调休是很正常的，而且通常不知道两年后的这三段时间的具体节假日怎么安排的，因此 `QuantLib` 中的 `ql.China()` 需要根据国务院办公厅的最新节假日安排来客制化。 对于 2021 年节假日，请参考网站 http://www.gov.cn/zhengce/content/2020-11/25/content_5564127.htm

---
### <div style="text-align: center"><font color='#dc2624' face='微软雅黑'>国务院办公厅关于 2021 年</font></div>
### <div style="text-align: center"><font color='#dc2624' face='微软雅黑'>部分节假日安排的通知</font></div>
### <div style="text-align: center"><font color='#dc2624' face='微软雅黑'>国办发明电〔2020〕27号</font></div>
<font color='#dc2624' face='微软雅黑'>
各省、自治区、直辖市人民政府，国务院各部委、各直属机构：经国务院批准，现将 2021 年元旦、春节、清明节、劳动节、端午节、中秋节和国庆节放假调休日期的具体安排通知如下。
<br>    

- 一、元旦：2021 年 1 月 1 日至 3 日放假，共 3 天。
- 二、**春节：2 月 11 日至 17 日放假调休，共 7 天。2 月 7 日（星期日）、2 月 20 日（星期六）上班。**
- 三、清明节：4 月 3 日至 5 日放假调休，共 3 天。
- 四、劳动节：5 月 1 日至 5 日放假调休，共 5 天。4 月 25 日（星期日）、5 月 8 日（星期六）上班。
- 五、端午节：6 月 12 日至 14 日放假，共 3 天。
- 六、中秋节：9 月 19 日至 21 日放假调休，共 3 天。9 月 18 日（星期六）上班。
- 七、国庆节：10 月 1 日至 7 日放假调休，共 7 天。9 月 26 日（星期日）、10 月 9 日（星期六）上班。
</font>
---


In [121]:
import re

CALENDAR_MAP = dict({
    'USNY': 'ql.UnitedStates()',
    'NY': 'ql.UnitedStates()',
    'USD': 'ql.UnitedStates()',
    
    'EUTA': 'ql.TARGET()', 
    'TARGET': 'ql.TARGET()', 
    'EUR': 'ql.TARGET()', 
    
    'GBLO': 'ql.UnitedKingdom()', 
    'LO': 'ql.UnitedKingdom()', 
    'GBP': 'ql.UnitedKingdom()', 
    
    'JPTO': 'ql.Japan()', 
    'TO': 'ql.Japan()',
    'JPY': 'ql.Japan()', 
        
    'AUSY': 'ql.Australia()', 
    'SY':'ql.Australia()',
    'AUD':'ql.Australia()', 
    
    'HKHK': 'ql.HongKong()', 
    'HK': 'ql.HongKong()', 
    'HKD': 'ql.HongKong()',
    
    'SGSG': 'ql.Singapore()', 
    'SG': 'ql.Singapore()', 
    'SGD': 'ql.Singapore()',
    
    'KRSE':'ql.SouthKorea()', 
    'SE':'ql.SouthKorea()', 
    'KRW':'ql.SouthKorea()', 
    
    'TWTA': 'ql.Taiwan()',
    'TA': 'ql.Taiwan()',
    'TWD':'ql.Taiwan()',
    
    'IDJA': 'ql.Indonesia()',
    'JA': 'ql.Indonesia()',
    'IDR':'ql.Indonesia()',
    
    'CNBJ': 'get_china_calendar()',
    'BJ': 'get_china_calendar()',
    'CNY': 'get_china_calendar()',
})


def get_china_calendar():
    cn_cal = ql.China()
    # 根据每年国务院发布部分节假日安排的通知来添加或删除中国节假日
    # 2021 - http://www.gov.cn/zhengce/content/2020-11/25/content_5564127.htm
    
    ########
    # 2021 #
    ########
    
    # 元旦：2021 年 1 月 1 日至 3 日放假，共 3 天。
    cn_cal.addHoliday( ql.Date(1,1,2021) )
    cn_cal.addHoliday( ql.Date(2,1,2021) )
    cn_cal.addHoliday( ql.Date(3,1,2021) )
    # 春节：2 月 11 日至 17 日放假调休，共 7 天。2 月 7 日（星期日）、2 月 20 日（星期六）上班。
    cn_cal.addHoliday( ql.Date(11,2,2021) )
    cn_cal.addHoliday( ql.Date(12,2,2021) )
    cn_cal.addHoliday( ql.Date(13,2,2021) )
    cn_cal.addHoliday( ql.Date(14,2,2021) )
    cn_cal.addHoliday( ql.Date(15,2,2021) )
    cn_cal.addHoliday( ql.Date(16,2,2021) )
    cn_cal.addHoliday( ql.Date(17,2,2021) )
    cn_cal.removeHoliday( ql.Date(7,2,2021) )
    cn_cal.removeHoliday( ql.Date(20,2,2021) )
    # 清明节：4 月 3 日至 5 日放假调休，共 3 天。
    cn_cal.addHoliday( ql.Date(3,4,2021) )
    cn_cal.addHoliday( ql.Date(4,4,2021) )
    cn_cal.addHoliday( ql.Date(5,4,2021) )
    # 劳动节：5 月 1 日至 5 日放假调休，共 5 天。4 月 25 日（星期日）、5 月 8 日（星期六）上班。
    cn_cal.removeHoliday( ql.Date(25,4,2021) )
    cn_cal.addHoliday( ql.Date(1,5,2021) )
    cn_cal.addHoliday( ql.Date(2,5,2021) )
    cn_cal.addHoliday( ql.Date(3,5,2021) )
    cn_cal.addHoliday( ql.Date(4,5,2021) )
    cn_cal.addHoliday( ql.Date(5,5,2021) )
    cn_cal.removeHoliday( ql.Date(8,5,2021) )
    # 端午节：6 月 12 日至 14 日放假，共 3 天。
    cn_cal.addHoliday( ql.Date(12,6,2021) )
    cn_cal.addHoliday( ql.Date(13,6,2021) )
    cn_cal.addHoliday( ql.Date(14,6,2021) )    
    # 中秋节：9 月 19 日至 21 日放假调休，共 3 天。9 月 18 日（星期六）上班。
    cn_cal.removeHoliday( ql.Date(18,9,2021) )
    cn_cal.addHoliday( ql.Date(19,9,2021) )
    cn_cal.addHoliday( ql.Date(20,9,2021) )
    cn_cal.addHoliday( ql.Date(21,9,2021) )
    # 国庆节：10 月 1 日至 7 日放假调休，共 7 天。9 月 26 日（星期日）、10 月 9 日（星期六）上班。
    cn_cal.removeHoliday( ql.Date(26,9,2021) )
    cn_cal.addHoliday( ql.Date(1,10,2021) )
    cn_cal.addHoliday( ql.Date(2,10,2021) )
    cn_cal.addHoliday( ql.Date(3,10,2021) )
    cn_cal.addHoliday( ql.Date(4,10,2021) )
    cn_cal.addHoliday( ql.Date(5,10,2021) )
    cn_cal.addHoliday( ql.Date(6,10,2021) )
    cn_cal.addHoliday( ql.Date(7,10,2021) )
    cn_cal.removeHoliday( ql.Date(9,10,2021) )
    
    return cn_cal

定义函数 `get_calendar()`，参数 `code` 可以是以上三种代号方式，输出是日历。有时候在跨货币市场上，现金流日需要发生在多个国家或地区上的工作日，比如 `USNY`、`EUTA` 和 `GBLO`。

定义函数 `get_calendars()`，参数 `codes` 可以是以上三种代号方式联合字符串，可用`空格 _ - ; ,` 链接，输出是联合日历。

In [122]:
def get_calendar( code ):
    c = code.upper().replace(' ', '')
    return eval(CALENDAR_MAP[c])

def get_calendars( codes ):
    code_list = re.split('\s|_|-|; |, |\*|\n', codes)
    if len(code_list) == 1:
        return get_calendar(code_list[0])
    else:
        calendars = [ get_calendar(c) for c in code_list ]
        return ql.JointCalendar( *calendars )

创建中国日历，该日历可用于 FR007 掉期产品。

In [123]:
cn_cal = get_calendar('CNBJ')

print( cn_cal.isHoliday(ql.Date(7,2,2021)) )    # 2 月 7 日（星期日）上班
print( cn_cal.isHoliday(ql.Date(11,2,2021)) )   # 农历新年
print( cn_cal.isHoliday(ql.Date(15,2,2021)) )   # 农历新年
print( cn_cal.isHoliday(ql.Date(20,2,2021)) )   # 2 月 20 日（星期六）上班。

False
True
True
False


常见美国和英国日历，该日历可用于生成 USD LIBOR 到期日。

In [39]:
us_eu_cal = get_calendars('USNY GBLO')
us_eu_cal

<QuantLib.QuantLib.JointCalendar; proxy of <Swig Object of type 'QuantLib::JointCalendar *' at 0x0000029E97DE7450> >

日期和日历基础打完之后，可以生成 FR007 掉期产品中所有相关日期了，主要分两大类：

- **产品相关日期**：
    - **生效日** - 通常从交易证实中获得
    - **到期日** - 通常从交易证实中获得
    - **每期起始日** - 用于计算利息
    - **每期终止日** - 用于计算利息
    - **支付日** - 现金流发生的日期，有时产品会有支付延迟特征


- **指标相关日期**
    - **FR007 每段起始日**  无调整的一组日期，用于计算利息
    - **FR007 每段终止日** - FR007 每段起始日加 7 个日历日，无调整的一组日期，用于计算利息
    - **FR007 定盘日** - FR007 每段起始日前一天工作日，一组日期
    - **FR007 指标起始日** - FR007 每段起始日顺延调整的工作日 (一组日期)
    - **FR007 指标终止日** - FR007 指标起始日加 7 个日历日，再做顺延调整的工作日 (一组日期)
    
### <font color='red' face='微软雅黑'> 后两节所有代码都来自 `schedule.py` 文件，包含生成金融产品和指标日期表的函数。</font>

### <font color='#2b4750' face='微软雅黑'>3.2 产品日期</font><a name='3.2'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#3)

假设要生成一组日期 <font color='blue'>${D_0, D_1, \cdots, D_n}$</font>，首先取定滚动日（RollDate)，如果要

- 前向生成日期，那么滚动日为起始日：<font color='blue'>$D_i = D_0 + i\cdot \text{Term}, i = 1,2,\cdots,n$</font>
- 后向生成日期，那么滚动日为终止日：<font color='blue'>$D_{n-i} = D_n - i\cdot \text{Term}, i = 1,2,\cdots,n$</font>

上式中，Term 是按计息频率设定的，例如对于半年付频率 Semiannual，Term 为六个月（6M）；对于季付 Quarter，Term 为三个月（3M）等等。上述过程可视图如下：

<img src="forward and backward date generation.PNG" width=60% height=60%>

得到一组日期后，需要做工作日调整（P, F, MP, MF），MF 最常见，此外如果要做月末修正（end of month），则要将所有（起始日除外）的日期都要移到月末。

该日期序列有 n+1 的日期，可分为

- n 个计息开始日 <font color='blue'>$D_{\text{start}}={D_0, D_1, \cdots, D_{n-1}}$</font>
- n 个计息结束日 <font color='blue'>$D_{\text{end}}={D_1, D_2, \cdots, D_n}$</font>

支付日期系列通常等于计息结束日系列，但对于有些产品的支付延迟不等于 0，那么需要在计息结束日上顺延，比如 a 天，直到落在工作日上。

- n 个支付日 <font color='blue'>$D_{\text{pay}}=D_{\text{end}} + a$</font>

---
计息频率可将一段时间划分成若干固定长度的时间段。如果这段时间是周期频率的整数倍，那么该日期表属于规则（regular）日期表，否则会剩余若干天数要分配给第一期和（或）最后一期，而这样的日期表属于不规则（irregular）日期表，或称为有存根期（stub period）的日期表。以 3 个月为周期频率举例：

- 一个 24 个月（2 年）的时期可以分为八个 3 个月，该日期表是规则的
- 一个 23 个月的时期不能分为整数倍的 3 个月，该日期表是不规则的，剩余的 2 个月可以

    - 全部安排到前端（用后向生成日期方法，不规则端会归在前端）
    - 全部安排到末端（用前向生成日期方法，不规则端会归在后端）
    - 分别安排到两端，只要总长度加起来等于 2 个月即可（需要额外定义规则段的首尾两个日期）
    
再着，前端和后端还分为短、长和灵活三种类型

- “短”指的是不规则计息区间小于规则计息区间
- “长”指的是不规则计息区间大于规则计息区间
- “灵”活指的是如果是不规则计息区间小于 7 天，合并成“长”，反之保持“短”

<img src="stub period.PNG" width=80% height=80%>

考虑以上所有特征，可以编写出计算日期表的函数 `date_series()`，其参数为

- `effectiveDate`：起始日，`ql.Date()` 格式，比如利率掉期的起始日 (effective date)
- `terminateDate`：终止日，`ql.Date()` 格式，比如利率掉期的到期日 (maturity date)
- `frequency`：付息频率，字符串格式，比如 'annual', 'semiannual', 'quarter' 等
- `calendar`：日历名，字符串格式，比如 'USD', 'GBLO', 'USNY EUTA' 等 
- `convention`：工作日惯例，字符串格式，有 'F', 'MF', 'P', 'MP', 'U' 五种，**<font color='blue'>默认为 MF</font>**
- `end_date_convention`：到期日的工作日惯例，字符串格式，有 'F', 'MF', 'P', 'MP', 'U' 五种，分别代表顺延、修正顺延、提前、修正提前和不调整，**<font color='blue'>默认为 MF</font>**
- `direction`：生成日期的方向，字符串格式，有 'Forward' 和 'Backward' 两种，**<font color='blue'>默认为 'Forward'</font>**
- `end_of_month`：月终惯例，布尔格式，有 **`True`** 和 **`False`** 两种，**<font color='blue'>默认为 `True`</font>**
- `stubConvention`：存根期惯例，字符串或 **`None`** 格式，字符串有 'both', 'short final','long final','smart final','short initial','long initial','smart initial' 七种，**<font color='blue'>默认为 `None`</font>**
- `firstRegularDate`：不规则前端日期，`ql.Date()` 格式或 **`None`** 格式，**<font color='blue'>默认为 `None`</font>**
- `lastRegularDate`：不规则末端日期，`ql.Date()` 格式或 **`None`** 格式，**<font color='blue'>默认为 `None`</font>**

In [40]:
def date_series( effectiveDate, terminateDate, frequency, calendar,
                 convention='MF', end_date_convention='MF',direction='Forward', EOM=True, 
                 stubConvention=None, firstRegularDate=None, lastRegularDate=None):
    # stubConvention = None, 'both', 'short final','long final','smart final','short initial','long initial','smart initial'
    
    date_s = effectiveDate
    date_e = terminateDate
    freq = ql_freq(frequency)
    calendar = get_calendars(calendar)
    bdc = ql_bdc(convention)
    end_bdc = ql_bdc(end_date_convention)
    direc = ql_dir(direction)
    
    stub = stubConvention if stubConvention is None else stubConvention.lower() 
    
    if stub is None:
        schedule = ql.Schedule( date_s, date_e, freq, calendar, bdc, end_bdc, direc, EOM )
    else:
        if stub in ('short final', 'long final', 'smart final'):
            schedule = ql.Schedule( date_s, date_e, freq, calendar, bdc, end_bdc, ql_dir('forward'), EOM )
        elif stub in ('short initial', 'long initial', 'smart initial'):
            schedule = ql.Schedule( date_s, date_e, freq, calendar, bdc, end_bdc, ql_dir('backward'), EOM )
        elif stub == 'both':
            date_first = asqldate(firstRegularDate)
            date_last = asqldate(lastRegularDate)
            schedule = ql.Schedule( date_s, date_e, freq, calendar, bdc, end_bdc, direc, EOM, date_first, date_last )
    
    schedule_list = list(schedule)
    
    if stub == 'long final':
        schedule_list.pop(-2)
    elif stub == 'long initial':
        schedule_list.pop(1)
    elif stub == 'smart final': # equivalent of LONG_FINAL up to 7 days and SHORT_FINAL beyond that
        # The schedule periods will be determined forwards from the regular period start date.
        # Any remaining period, shorter than the standard frequency, will be allocated at the end.
        # If this results in a stub of less than 7 days, the stub will be combined with the next period.
        # If this results in a stub of 7 days or more, the stub will be retained.
        diff = schedule_list[-1] - schedule_list[-2]
        if diff < 7:
            schedule_list.pop(-2)
    elif stub == 'smart initial': # equivalent of LONG_INITIAL up to 7 days and SHORT_INITIAL beyond that. 
        # The schedule periods will be determined backwards from the regular period end date. 
        # Any remaining period, shorter than the standard frequency, will be allocated at the start. 
        # If this results in a stub of less than 7 days, the stub will be combined with the next period. 
        # If this results in a stub of 7 days or more, the stub will be retained. 
        diff = schedule_list[1] - schedule_list[0]
        if diff < 7:
            schedule_list.pop(1)
 
    output = dict(start_dates=schedule_list[:-1], end_dates=schedule_list[1:], all_dates=schedule_list )
    return output

从上面代码可看出用到了 `QuantLib` 里的核心函数 `ql.Schedule()`,

    schedule = ql.Schedule( date_s, date_e, freq, calendar, bdc, end_bdc, direc, EOM )

除了 `date_s` 和 `date_e` 两个是日期之外，后面五个参数分别是 `QuantLib` 中的频率 (`freq`)、日历 (`calendar`)、工作日惯例 (`bdc`, `end_bdc`) 和方向 (`direc`) 对象，但是从函数 `date_series()` 传入的参数格式都是字符串，因此需要做一些转换：

    freq = ql_freq(frequency)
    calendar = get_calendars(calendar)
    bdc = ql_bdc(convention)
    end_bdc = ql_bdc(end_date_convention)
    direc = ql_dir(direction)
    
### <font color='black' face='微软雅黑'>函数 `ql_freq()`：从频率字符串到 QuantLib 对象</font>

In [41]:
def ql_freq( frequency ):
    f = frequency.lower()[0]
    return ql.Period(1, ql.Years) if f == 'a' else \
           ql.Period(6, ql.Months) if f == 's' else \
           ql.Period(3, ql.Months) if f == 'q' else \
           ql.Period(1, ql.Months) if f == 'm' else \
           ql.Period(2, ql.Weeks) if f == 'b' else \
           ql.Period(1, ql.Weeks) if f == 'w' else \
           ql.Period(1, ql.Days)

In [124]:
print( ql_freq('annually') )
print( ql_freq('semi-annually') )
print( ql_freq('quarterly') )
print( ql_freq('monthly') )
print( ql_freq('bi-weekly') )
print( ql_freq('weekly') )
print( ql_freq('daily') )

1Y
6M
3M
1M
2W
1W
1D


### <font color='black' face='微软雅黑'>函数 `ql_bdc()`：从工作日惯例字符串到以数字代表的 QuantLib 对象</font>
日期惯例的五种：
1. `ql.Following (0)`：顺延
2. `ql.ModifiedFollowing (1)`：修正顺延
3. `ql.Preceding (2)`：提前
4. `ql.ModifiedPreceding (3)`：修正提前
5. `ql.Unadjusted (4)`：不调整

In [43]:
def ql_bdc( convention ):
    BDC = convention.lower().replace(' ','')
    if BDC in ('following', 'f'):
        qlbdc = ql.Following
    elif BDC in ('modifiedfollowing','mf'):
        qlbdc = ql.ModifiedFollowing
    elif BDC in ('preceding', 'p'):
        qlbdc = ql.Preceding
    elif BDC in ('modifiedpreceding','mp'):
        qlbdc = ql.ModifiedPreceding
    elif BDC in ('unadjusted', 'u'):
        qlbdc = ql.Unadjusted
    return qlbdc

In [125]:
print( ql_bdc('F') )
print( ql_bdc('MF') )
print( ql_bdc('P') )
print( ql_bdc('MP') )
print( ql_bdc('U') )

0
1
2
3
4


### <font color='black' face='微软雅黑'>函数 `ql_dir()`：从日期生成方向字符串到以数字代表的 QuantLib 对象</font>

生成方向的两种：
1. `ql.DateGeneration.Backward (0)`：后向
2. `ql.DateGeneration.Forward (1)`：前向

In [45]:
def ql_dir( direction ):
    direc = direction.lower()[0]
    if direc == 'b':
        qldir = ql.DateGeneration.Backward
    elif direc == 'f':
        qldir = ql.DateGeneration.Forward
    return qldir

In [46]:
print( ql_dir('Backward') )
print( ql_dir('Forward') )

0
1


用 `date_series()` 函数来生成第一笔 FR007 掉期的固定端的日期表。

In [47]:
deal = INSTRUMENT['IR Compound IRS']['CMPIRS1001']
deal

ID
Effective Date        20-Jan-21
Maturity Date         20-Jan-23
Notional              100000000
Currency                    CNY
DiscountCurve             FR007
Fixed Leg                   Pay
Fixed Frequency         Quarter
Fixed Daycount          act/365
Rate                        2.4
Floating Leg            Receive
Floating Frequency      Quarter
Floating Daycount       act/365
ReferenceCurve            FR007
Spread                        0
Payment Calendar           CNBJ
Fixing Calendar            CNBJ
BDC                          MF
Name: CMPIRS1001, dtype: object

In [48]:
inst_schedule = date_series( asqldate(deal['Effective Date']), asqldate(deal['Maturity Date']), 
                             deal['Fixed Frequency'], deal['Payment Calendar'], deal['BDC'] )
pd.DataFrame( {k:v for k, v in inst_schedule.items() if k !='all_dates'} )

,start_dates,end_dates
0,"January 20th, 2021","April 20th, 2021"
1,"April 20th, 2021","July 20th, 2021"
2,"July 20th, 2021","October 20th, 2021"
3,"October 20th, 2021","January 20th, 2022"
4,"January 20th, 2022","April 20th, 2022"
5,"April 20th, 2022","July 20th, 2022"
6,"July 20th, 2022","October 20th, 2022"
7,"October 20th, 2022","January 20th, 2023"


### <font color='#2b4750' face='微软雅黑'>3.3 指标日期</font><a name='3.3'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#3)

具体来说，在第 <font color='blue'>$n$</font> 期 <font color='blue'>$[T_{n-1}, T_{n}]$</font>，七天利率的年限为 1 周，重置工作日规则为**不调整**，即每次添加 7 个日历日 (7D) 而不做任何调整。

<br>
<font color='blue'>
\begin{align}
t_{n,k} &= T_{n-1} + k \cdot 7D, (k=0,1,2,⋯,n_{k}) \\
t_{n,n_k} &= T_{n}
\end{align}
</font>

因为 <font color='blue'>$t_{n,n_k} = T_{n}$</font>，而每个付息期限（按季度或按半年）可能不是周的倍数，期限末的最后一个复利期可能会小于 7 天。

上述描述可视如下：

<img src="7D Rate Detail.png" width=65% height=65%>

具体做法如下：

1. 先确定 <font color='blue'>$t_{n,k}$</font>：从第 <font color='blue'>$i$</font> 期起始日开始做为 <font color='blue'>$t_{n,0}$</font>，在 <font color='blue'>$t_{n,0}$</font> 基础上加 7 天得到 <font color='blue'>$t_{n,1}$</font>，以此类推得到 <font color='blue'>$t_{n,2}, t_{n,3}, \cdots, t_{n,n_k}$</font>。


2. 计算 7D 利率的起始日和结束日：对于第 <font color='blue'>$k$</font> 个利率，首先对 <font color='blue'>$t_{n,k}$</font> 使用**修正顺延**惯例调整，得到 <font color='blue'>$s_{n,k}$</font> 作为利率的起始日，接着在 <font color='blue'>$s_{n,k}$</font> 的基础上往后推 7 天得到 <font color='blue'>$e_{n,k}$</font>，同样使用**修正顺延**惯例调整。

考虑以上所有特征，可以编写出计算日期表的函数 `CMPR_date()`，其参数为:

- `start_date`：每期的起始日，`ql.Date()` 格式
- `end_date`：每期的终止日，`ql.Date()` 格式
- `curr`：币种，字符串格式，比如 'CNY', 'USD', 'EUR' 等
- `index`：指标名称，字符串格式，比如 'FR 7D', 'FDR 14D' 等
- `fixing_calendar`：日历名，字符串格式，比如 'CNBJ', 'GBLO' 等 
- `bdc`：工作日惯例，字符串格式，有 'F', 'MF', 'P', 'MP', 'U' 五种，**<font color='blue'>默认为 MF</font>**
- `direc`：生成日期的方向，字符串格式，有 'Forward' 和 'Backward' 两种，**<font color='blue'>默认为 'Forward'</font>**

In [49]:
def CMPR_date( start_date, end_date, curr, index, fixing_calendar, bdc='MF', direc='Forward' ):
    
    cal = get_calendars(fixing_calendar)
    day_lag = swap_day_lag(curr)
    CMPR_tenor = ql_period(index)
    
    period_start_date = []
    period_end_date = []
    CMPR_fixing_date = []
    CMPR_start_date = []
    CMPR_end_date = []

    for Ds, De in zip(start_date, end_date):
        schedule = ql.MakeSchedule( Ds, De, tenor=CMPR_tenor, rule=ql_dir(direc) )

        period_start, period_end = list(schedule)[:-1], list(schedule)[1:]
        
        CMPR_start = [ cal.adjust( d, ql_bdc(bdc)) for d in period_start ] 
        CMPR_end = [ cal.adjust(d+CMPR_tenor, ql_bdc(bdc)) for d in CMPR_start ]
        CMPR_fixing = [ cal.advance(d, ql.Period(-day_lag, ql.Days)) for d in CMPR_start ]
        
        period_start_date.append( period_start )
        period_end_date.append( period_end )
        CMPR_start_date.append( CMPR_start )
        CMPR_end_date.append( CMPR_end )
        CMPR_fixing_date.append( CMPR_fixing )
    
    output = dict(period_start_dates=period_start_date,
                  period_end_dates=period_end_date,
                  index_fixing_dates=CMPR_fixing_date, 
                  index_start_dates=CMPR_start_date, 
                  index_end_dates=CMPR_end_date)
    return output

上面代码第 4 和 5 行分别得到

- 人民币掉期市场的即期延迟，用函数 `swap_day_lag()` 来实现
- 期限字符串转换成 QuantLib Period 对象，用 `ql_period()` 来实现

In [50]:
def swap_day_lag( curr ):
    curr = curr.upper()
    if curr in ('GBP', 'HKD'):
        day_lag = 0
    elif curr in ('CNY', 'KRW', 'AUD', 'CAD'):
        day_lag = 1
    else:
        day_lag = 2
    return day_lag

### <font color='black' face='微软雅黑'>函数 `ql_period()`：从期限字符串到 QuantLib 对象</font>

In [51]:
def ql_period( ticker ):
    # ticker 可以是 3M,  LIBOR3M, LIBOR-3M, LIBOR_3M
    ticker = re.split('\s|_|-', curve_name(ticker))[-1]
    unit = ticker[-1]
    no_of_unit = int(ticker[:-1])
    
    unit_obj = ql.Years if unit == 'Y' else \
               ql.Months if unit == 'M' else \
               ql.Weeks if unit == 'W' else \
               ql.Days
    
    return ql.Period( no_of_unit, unit_obj )


def curve_name( curve_name ):
    curve_name = curve_name.upper()
    MAP = {'FR001':'FR 1D', 'FR007':'FR 7D', 'FR014':'FR 14D', 
           'FDR001':'FR 1D', 'FDR007':'FDR 7D', 'FDR014':'FDR 14D'}
    if curve_name in MAP.keys():
        return MAP[curve_name]
    else:
        return curve_name

In [52]:
print( ql_period('FR 7D') )
print( ql_period('LIBOR 3M') )
print( ql_period('SOFR 1D') )

1W
3M
1D


In [53]:
start_date = inst_schedule['start_dates']
end_date = inst_schedule['end_dates']
curr = 'CNY'
index = 'FR 7D'
fixing_calendar = 'CNBJ'

index_schedule = CMPR_date( start_date, end_date, curr, index, fixing_calendar )
df =pd.DataFrame( index_schedule )
df

,period_start_dates,period_end_dates,index_fixing_dates,index_start_dates,index_end_dates
0,"[January 20th, 2021, January 27th, 2021, Febru...","[January 27th, 2021, February 3rd, 2021, Febru...","[January 19th, 2021, January 26th, 2021, Febru...","[January 20th, 2021, January 27th, 2021, Febru...","[January 27th, 2021, February 3rd, 2021, Febru..."
1,"[April 20th, 2021, April 27th, 2021, May 4th, ...","[April 27th, 2021, May 4th, 2021, May 11th, 20...","[April 19th, 2021, April 26th, 2021, April 30t...","[April 20th, 2021, April 27th, 2021, May 6th, ...","[April 27th, 2021, May 6th, 2021, May 13th, 20..."
2,"[July 20th, 2021, July 27th, 2021, August 3rd,...","[July 27th, 2021, August 3rd, 2021, August 10t...","[July 19th, 2021, July 26th, 2021, August 2nd,...","[July 20th, 2021, July 27th, 2021, August 3rd,...","[July 27th, 2021, August 3rd, 2021, August 10t..."
3,"[October 20th, 2021, October 27th, 2021, Novem...","[October 27th, 2021, November 3rd, 2021, Novem...","[October 19th, 2021, October 26th, 2021, Novem...","[October 20th, 2021, October 27th, 2021, Novem...","[October 27th, 2021, November 3rd, 2021, Novem..."
4,"[January 20th, 2022, January 27th, 2022, Febru...","[January 27th, 2022, February 3rd, 2022, Febru...","[January 19th, 2022, January 26th, 2022, Febru...","[January 20th, 2022, January 27th, 2022, Febru...","[January 27th, 2022, February 3rd, 2022, Febru..."
5,"[April 20th, 2022, April 27th, 2022, May 4th, ...","[April 27th, 2022, May 4th, 2022, May 11th, 20...","[April 19th, 2022, April 26th, 2022, May 3rd, ...","[April 20th, 2022, April 27th, 2022, May 4th, ...","[April 27th, 2022, May 4th, 2022, May 11th, 20..."
6,"[July 20th, 2022, July 27th, 2022, August 3rd,...","[July 27th, 2022, August 3rd, 2022, August 10t...","[July 19th, 2022, July 26th, 2022, August 2nd,...","[July 20th, 2022, July 27th, 2022, August 3rd,...","[July 27th, 2022, August 3rd, 2022, August 10t..."
7,"[October 20th, 2022, October 27th, 2022, Novem...","[October 27th, 2022, November 3rd, 2022, Novem...","[October 19th, 2022, October 26th, 2022, Novem...","[October 20th, 2022, October 27th, 2022, Novem...","[October 27th, 2022, November 3rd, 2022, Novem..."


In [54]:
idx = 2
fixing = df.loc[idx,'index_fixing_dates']
prd_start = df.loc[idx,'period_start_dates']
prd_end = df.loc[idx,'period_end_dates']
index_start = df.loc[idx,'index_start_dates']
index_end = df.loc[idx,'index_end_dates']

pd.DataFrame( {'定盘日':fixing, '期开始日':prd_start, '期结束日':prd_end, '利率起始日':index_start, '利率结束日':index_end} )

,定盘日,期开始日,期结束日,利率起始日,利率结束日
0,"July 19th, 2021","July 20th, 2021","July 27th, 2021","July 20th, 2021","July 27th, 2021"
1,"July 26th, 2021","July 27th, 2021","August 3rd, 2021","July 27th, 2021","August 3rd, 2021"
2,"August 2nd, 2021","August 3rd, 2021","August 10th, 2021","August 3rd, 2021","August 10th, 2021"
3,"August 9th, 2021","August 10th, 2021","August 17th, 2021","August 10th, 2021","August 17th, 2021"
4,"August 16th, 2021","August 17th, 2021","August 24th, 2021","August 17th, 2021","August 24th, 2021"
5,"August 23rd, 2021","August 24th, 2021","August 31st, 2021","August 24th, 2021","August 31st, 2021"
6,"August 30th, 2021","August 31st, 2021","September 7th, 2021","August 31st, 2021","September 7th, 2021"
7,"September 6th, 2021","September 7th, 2021","September 14th, 2021","September 7th, 2021","September 14th, 2021"
8,"September 13th, 2021","September 14th, 2021","September 21st, 2021","September 14th, 2021","September 22nd, 2021"
9,"September 18th, 2021","September 21st, 2021","September 28th, 2021","September 22nd, 2021","September 29th, 2021"


注意**期起始日**和**期结束日**之间都只相差 7 天，除了最后一期相差 1 天。

**利率起始日**和**利率结束日**都是工作日，比如第 10 行的 2021 年 10 月 5 日是国庆，因此利率起始日调整到 2021 年 10 月 8 日，再加上 7 天得到 2021 年 10 月 15 日。

将<font color='red'>**产品日期**</font>和<font color='red'>**指标日期**</font>合并在一起，得到为 FR007 掉期类产品的日期表生成函数 `CMPIRS_schedule()`。

In [55]:
def CMPIRS_schedule( date_eff, date_mat, curr, index, fix_freq, flt_freq, pay_cal, fixing_cal, 
                     fix_pay_delay=0, flt_pay_delay=0, bdc='MF', end_date_bdc='MF',direc='Forward', EOM=True, 
                     stub=None, date_first=None, date_last=None):
    
    fix_schedule = date_series(date_eff, date_mat, fix_freq, pay_cal, bdc, end_date_bdc, direc, EOM, stub, date_first, date_last)
    flt_schedule = date_series(date_eff, date_mat, flt_freq, pay_cal, bdc, end_date_bdc, direc, EOM, stub, date_first, date_last)
    
    CMPR_schedule = CMPR_date( flt_schedule['start_dates'], flt_schedule['end_dates'], curr, index, fixing_cal )
    fix_pay_dates = shift_date( fix_schedule['end_dates'], pay_cal, fix_pay_delay )
    flt_pay_dates = shift_date( flt_schedule['end_dates'], pay_cal, flt_pay_delay )
    
    fix_schedule['payment_dates'] = fix_pay_dates
    flt_schedule['payment_dates'] = flt_pay_dates
    
    flt_schedule = {**flt_schedule, **CMPR_schedule}
    
    output = {'fixed leg schedule': fix_schedule, 'floating leg schedule': flt_schedule}
    return output

上面代码第 9 和 10 行分别用函数 `shift_date()` 将固定端和浮动端的支付日分别移动 `fix_pay_delay` 和 `flt_pay_delay` 个单位。

In [56]:
def shift_date( dates, calendar, delay=0 ):
    if delay == 0:
        shift_dates = dates
    else:
        cal = get_calendars(calendar)
        shift_dates = [ cal.advance(d, ql.Period(delay,ql.Days)) for d in dates ]
    
    return shift_dates

最后使用函数 `CMPIRS_schedule()` 生成第一笔 FR007 掉期完整日期表。

In [57]:
date_eff = asqldate(deal['Effective Date'])
date_mat = asqldate(deal['Maturity Date'])
fix_freq = deal['Fixed Frequency']
flt_freq = deal['Floating Frequency']
curr = deal['Currency']
index = deal['ReferenceCurve']
pay_cal = deal['Payment Calendar']
fixing_cal = deal['Fixing Calendar']

schedule = CMPIRS_schedule( date_eff, date_mat, curr, index, fix_freq, flt_freq, pay_cal, fixing_cal )

打印出其固定端的日期表。

In [58]:
CMPIRS_fixed_leg_schedule = pd.DataFrame( {k:v for k,v in schedule['fixed leg schedule'].items() if k !='all_dates'} )
CMPIRS_fixed_leg_schedule

,start_dates,end_dates,payment_dates
0,"January 20th, 2021","April 20th, 2021","April 20th, 2021"
1,"April 20th, 2021","July 20th, 2021","July 20th, 2021"
2,"July 20th, 2021","October 20th, 2021","October 20th, 2021"
3,"October 20th, 2021","January 20th, 2022","January 20th, 2022"
4,"January 20th, 2022","April 20th, 2022","April 20th, 2022"
5,"April 20th, 2022","July 20th, 2022","July 20th, 2022"
6,"July 20th, 2022","October 20th, 2022","October 20th, 2022"
7,"October 20th, 2022","January 20th, 2023","January 20th, 2023"


打印出其浮动端的日期表。

In [59]:
CMPIRS_floating_leg_schedule = pd.DataFrame( {k:v for k,v in schedule['floating leg schedule'].items() if k !='all_dates'} )
CMPIRS_floating_leg_schedule

,start_dates,end_dates,payment_dates,period_start_dates,period_end_dates,index_fixing_dates,index_start_dates,index_end_dates
0,"January 20th, 2021","April 20th, 2021","April 20th, 2021","[January 20th, 2021, January 27th, 2021, Febru...","[January 27th, 2021, February 3rd, 2021, Febru...","[January 19th, 2021, January 26th, 2021, Febru...","[January 20th, 2021, January 27th, 2021, Febru...","[January 27th, 2021, February 3rd, 2021, Febru..."
1,"April 20th, 2021","July 20th, 2021","July 20th, 2021","[April 20th, 2021, April 27th, 2021, May 4th, ...","[April 27th, 2021, May 4th, 2021, May 11th, 20...","[April 19th, 2021, April 26th, 2021, April 30t...","[April 20th, 2021, April 27th, 2021, May 6th, ...","[April 27th, 2021, May 6th, 2021, May 13th, 20..."
2,"July 20th, 2021","October 20th, 2021","October 20th, 2021","[July 20th, 2021, July 27th, 2021, August 3rd,...","[July 27th, 2021, August 3rd, 2021, August 10t...","[July 19th, 2021, July 26th, 2021, August 2nd,...","[July 20th, 2021, July 27th, 2021, August 3rd,...","[July 27th, 2021, August 3rd, 2021, August 10t..."
3,"October 20th, 2021","January 20th, 2022","January 20th, 2022","[October 20th, 2021, October 27th, 2021, Novem...","[October 27th, 2021, November 3rd, 2021, Novem...","[October 19th, 2021, October 26th, 2021, Novem...","[October 20th, 2021, October 27th, 2021, Novem...","[October 27th, 2021, November 3rd, 2021, Novem..."
4,"January 20th, 2022","April 20th, 2022","April 20th, 2022","[January 20th, 2022, January 27th, 2022, Febru...","[January 27th, 2022, February 3rd, 2022, Febru...","[January 19th, 2022, January 26th, 2022, Febru...","[January 20th, 2022, January 27th, 2022, Febru...","[January 27th, 2022, February 3rd, 2022, Febru..."
5,"April 20th, 2022","July 20th, 2022","July 20th, 2022","[April 20th, 2022, April 27th, 2022, May 4th, ...","[April 27th, 2022, May 4th, 2022, May 11th, 20...","[April 19th, 2022, April 26th, 2022, May 3rd, ...","[April 20th, 2022, April 27th, 2022, May 4th, ...","[April 27th, 2022, May 4th, 2022, May 11th, 20..."
6,"July 20th, 2022","October 20th, 2022","October 20th, 2022","[July 20th, 2022, July 27th, 2022, August 3rd,...","[July 27th, 2022, August 3rd, 2022, August 10t...","[July 19th, 2022, July 26th, 2022, August 2nd,...","[July 20th, 2022, July 27th, 2022, August 3rd,...","[July 27th, 2022, August 3rd, 2022, August 10t..."
7,"October 20th, 2022","January 20th, 2023","January 20th, 2023","[October 20th, 2022, October 27th, 2022, Novem...","[October 27th, 2022, November 3rd, 2022, Novem...","[October 19th, 2022, October 26th, 2022, Novem...","[October 20th, 2022, October 27th, 2022, Novem...","[October 27th, 2022, November 3rd, 2022, Novem..."


# <font color='#dc2624' face='微软雅黑'>4. 变量计算</font><a name='4'></a>
[<font color='black' face='微软雅黑'>回到目录</font>](#toc)

### <font color='red' face='微软雅黑'> 本节所有代码都来自 `market_variable.py` 文件，专门用来计算市场变量。</font>

### <font color='#2b4750' face='微软雅黑'>4.1 折现因子</font><a name='4.1'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#4)

折现因子 (discount factor) 就是把**未来 $T$ 时点的现金流折算成 $t$ 时点的现值**的一个数。金融工程里通常用 $P(t,T)$ 来表示折现因子，当 $t=0$ 时，用 $P(T)$ 记作 $P(0,T)$。

下面公式写出连续复利下的折现因子 $P(T)$ 和利率 $r(T)$ 之间的关系。

<br>
<font color='blue'>
\begin{align}
P(T) = \text{exp}(-r(T)\cdot T)
\end{align}
</font>

In [60]:
import datetime
import numpy as np
import pandas as pd
import scipy.interpolate as spi

In [61]:
def get_discount( curve, date, today ):
    today = asdatetime(today)
    curve_date = pd.DatetimeIndex(curve.index)
    r_std = curve['Rate']
    d_std = (curve_date - today).days.values
    
    if isinstance(date, (list, np.ndarray)):
        date = asdatetimes(date)
        d = np.array( [(dd-today).days for dd in date] )
    else:
        date = asdatetime(date)
        d = (date - today).days
        
    f = spi.interp1d(d_std, r_std, fill_value=(r_std[0], r_std[-1]), bounds_error=False)
    r = f(d)
    DF = np.exp(-d/365 * r/100)
    return DF

函数 `get_discount()` 计算折现因子 $P(t,T)$, 三个参数分别是零息曲线 `curve`、大 $T$ `date` 和小 $t$ `today`。

现在假设已经用拔靴法构建出 FR007 的零息曲线，下一节再详细曲线构建过程。

In [126]:
from curve_construction import bootstrapping

curr, market_index, index = 'CNY', 'FR007', 'FR 7D'
curve = bootstrapping( MARKET, curr, market_index, date_eval )
curve

,Quote,Rate,Discount
Date,,,
2021-03-18,2.3087,2.306658,0.997728
2021-05-18,2.5909,2.559980,0.993220
2021-08-18,2.6475,2.624956,0.986500
2021-11-18,2.6763,2.657438,0.979749
2022-02-18,2.7100,2.692966,0.972855
2023-02-20,2.8139,2.800621,0.944802
2024-02-19,2.9068,2.896246,0.916136
2025-02-18,2.9901,2.982516,0.886961
2026-02-18,3.0807,3.077392,0.856806


In [127]:
pay_date = schedule['fixed leg schedule']['payment_dates']
discount = get_discount( curve, pay_date, date_eval )
discount

array([0.99539105, 0.98864805, 0.98188945, 0.97504199, 0.96828072,
       0.96138938, 0.95434303, 0.94721946])

In [128]:
df = pd.DataFrame( discount, index=asdatetimes(pay_date), columns=['value'] )
df.reset_index().iplot( x='index', y='value', mode='lines+markers', size=10, 
                        xTitle='日期', yTitle='折现因子',
                        title='FR007 掉期固定端的支付日折现因子', theme='solar')

### <font color='#2b4750' face='微软雅黑'>4.2 远期利率</font><a name='4.2'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#4)

上节的零息利率都是从 0 时点开始的利率，也称为即期利率 (spot rate)，而远期利率 (forward rate) 是隐含在给定的即期利率中从未来的某一时点到另一时点的利率水平。

计算远期利率的公式如下：

<br/>
<font color='blue'>
\begin{equation}
F(0,T_s,T_e) = \frac{1}{\tau}\left(\frac{P(0,T_s)}{P(0,T_e)}-1\right)
\end{equation}
</font>

其中
- $T_s$ = 起始日到估值日之间的年限 (使用 **act/365** 日期计数惯例)
- $T_e$ = 终止日到估值日之间的年限
- $\tau$ = 起始日和终止日之间的年限 (不同货币的日期计数惯例不一样，通常使用 **act/360** ) 
- $P(0,T_s)$ = 在 $T_s$ 时点上的折现因子
- $P(0,T_e)$ = 在 $T_e$ 时点上的折现因子

函数 `get_forward_rate()` 计算折现因子 $F(t,T)$, 五个参数分别是零息曲线 `curve`、起始日 `date_s`、终止日 `date_e`、观测远期利率日 `date_obs` 和天数计数惯例 `dcc`。

In [65]:
def get_forward_rate( curve, date_s, date_e, date_obs, dcc='act/360' ):
    if isinstance(date_s, (list, np.ndarray)):
        tau = np.array( [yearfrac(ds, de, dcc) for ds, de in zip(date_s, date_e)] )
    else:
        tau = yearfrac(date_s, date_e, dcc)
        
    DF_s = get_discount( curve, date_s, date_obs )
    DF_e = get_discount( curve, date_e, date_obs )
    F = (1/tau)*(DF_s/DF_e - 1)
    return F

上面代码第 3 和 5 行中的函数 `yearfrac()`，在给定一个日期计数惯例，用来计算两个日期或两组日期之间的年限。

### <font color='red' face='微软雅黑'> 下个单元格所有代码都来自 `daycount_utils.py` 文件，专门用来计算年限。</font>

In [66]:
_dc_norm = dict({
    '30/360 US': '30/360 US',
    '30U/360': '30/360 US',
    '360/360': '30/360 US',
    'BOND BASIS': '30/360 US',

    '30E/360': '30E/360',
    '30/360 ICMA': '30E/360',
    '30S/360 ICMA': '30E/360',
    'EUROBOND BASIS (ISDA 2006)': '30E/360',
    'SPECIAL GERMAN': '30E/360',

    '30E/360 ISDA': '30E/360 ISDA',
    'EUROBOND BASIS (ISDA 2000)': '30E/360 ISDA',
    'GERMAN': '30E/360 ISDA',

    '30E+/360': '30E+/360',

    'ACTUAL/ACTUAL ISDA': 'ACTUAL/ACTUAL ISDA',
    'ACTUAL/365': 'ACTUAL/ACTUAL ISDA',
    'ACT/365': 'ACTUAL/ACTUAL ISDA',
    'ACT/ACT': 'ACTUAL/ACTUAL ISDA',
    'ACTUAL/ACTUAL': 'ACTUAL/ACTUAL ISDA',

    'ACTUAL/365 FIXED': 'ACTUAL/365 FIXED',
    'ACT/365 FIXED': 'ACTUAL/365 FIXED',
    'A/365F': 'ACTUAL/365 FIXED',
    'ENGLISH': 'ACTUAL/365 FIXED',

    'ACTUAL/360': 'ACTUAL/360',
    'A/360': 'ACTUAL/360',
    'ACT/360': 'ACTUAL/360',
    'FRENCH': 'ACTUAL/360',

    'ACTUAL/365L': 'ACTUAL/365L',
    'ISMA-YEAR': 'ACTUAL/365L',

    'ACTUAL/ACTUAL AFB': 'ACTUAL/ACTUAL AFB',
    'ACT/ACT AFB': 'ACTUAL/ACTUAL AFB'
})

def _normalize_daycount_convention(convention):
    convention = convention.upper()
    return _dc_norm[convention]

def _period_has_29feb(dt1, dt2):
    have_29_feb = False
    y1 = dt1.year
    y2 = dt2.year
    for y in range(y1, y2+1):
        if leapyear(y) and (
            (y!=y1 and y!=y2)
            or (y == y1 and dt1<datetime.datetime(y1, 2, 29))
            or (y == y2 and datetime.datetime(y2, 2, 29) <= dt2)):
            have_29_feb = True
    return have_29_feb


def _daycount_parameters(dt1, dt2, convention, **kwargs):
    """ Return number of days and total number of days (i.e. numerator and
        denominator in the counting of year fraction between dates
    """
    convention = convention.upper()
    convention = _normalize_daycount_convention(convention)
    dt1 = asdatetime(dt1)
    dt2 = asdatetime(dt2)
    y1, m1, d1 = dt1.year, dt1.month, dt1.day
    y2, m2, d2 = dt2.year, dt2.month, dt2.day
    factor = None

    if convention in {'30/360 US', '30E/360', '30E/360 ISDA', '30E+/360'}:
        eom = 'eom' in kwargs and kwargs['eom']

        if convention == '30/360 US':
            # US adjustments
            if eom and dt1.month == 2 and iseom(dt1) and dt2.month == 2 and iseom(dt2):
                d2 = 30
            if eom and dt1.month == 2 and iseom(dt1):
                d1 = 30
            if d2 == 31 and d1>=30:
                d2 = 30
            if d1 == 31:
                d1 = 30
        elif convention == '30E/360':
            if d1 == 31:
                d1 = 30
            if d2 == 31:
                d2 = 30
        elif convention == '30E/360 ISDA':
            if iseom(dt1):
                d1 = 30
            if iseom(dt2) and m2 != 2:
                d2 = 30
        elif convention == '30E+/360':
            if d1 == 31:
                d1 = 30
            if d2 == 31:
                m2 += 1
                if m2 == 13:
                    m2 = 1
                    y2 += 1
                d2 = 1

        num_days = (360*(y2-y1)+30*(m2-m1)+(d2-d1))
        year_days = 360

    elif convention == 'ACTUAL/ACTUAL ISDA':
        num_days = 0
        year_days = 0

        if y2 == y1:
            num_days = (dt2 - dt1).days
            year_days = yeardays(y1)
        else:
            # we need to calculate factor properly
            factor = 0.0
            # full years between y1 and y2 exclusive
            for y in range(y1+1, y2):
                yd = yeardays(y)
                num_days += yd
                year_days += yd
                factor += float(num_days)/year_days
            # days in the remaining part of the first year
            num = (datetime.datetime(y1+1, 1, 1) - dt1).days
            den = yeardays(y1)
            num_days += num
            year_days += den
            factor += float(num)/den
            # days in the beginning of the last year
            num = (dt2 - datetime.datetime(y2, 1, 1)).days
            den = yeardays(y2)
            num_days += num
            year_days += den
            factor += float(num)/den

    elif convention == 'ACTUAL/365 FIXED':
        num_days = (dt2-dt1).days
        year_days = 365
    elif convention == 'ACTUAL/360':
        num_days = (dt2-dt1).days
        year_days = 360
    elif convention == 'ACTUAL/365L':
        yearly_frequency = 'frequency' in kwargs and kwargs['frequency'] =='yearly'
        if yearly_frequency:
            year_days = 366 if _period_has_29feb(dt1, dt2) else 365
        else:
            year_days = 366 if leapyear(dt2) else 365
        num_days = (dt2-dt1).days
    elif convention == 'ACTUAL/ACTUAL AFB':
        year_days = 366 if _period_has_29feb(dt1, dt2) else 365
        num_days = (dt2-dt1).days
    else:
        raise ValueError('Unknown daycount convention \'%s\'' % convention)

    if factor is None:
        factor = float(num_days)/year_days

    return num_days, year_days, factor

def yearfrac(dt1, dt2, convention, **kwargs):
    return _daycount_parameters(dt1, dt2, convention, **kwargs)[2]

以上函数用到其他基本函数，如 `iseom()`、`leapyear()` 和 `yeardays()`。

### <font color='red' face='微软雅黑'> 下个单元格所有代码都来自 `date_utils.py` 文件。</font>

In [67]:
DAYS_IN_WEEK = 7
MONTHS_IN_YEAR = 12
 
DAYS_IN_NON_LEAP_YEAR = 365
DAYS_IN_LEAP_YEAR = 366

LAST_DAY_IN_MONTH =  [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

def asyear(dt):
    """ Extract year value from integer, date string, datetime.date or datetime.datetime class
    """
    if isinstance(dt, int):
        return dt
    elif isinstance (dt, datetime.date) or isinstance(dt, datetime.datetime):
        return dt.year
    elif isinstance(dt, str):
        dt = asdatetime(dt)
        return dt.year
    else:
        raise ValueError('Cannot extract year value from %s', repr(dt))

def leapyear(dt):
    """ Check if year is a leap year
    """
    yr = asyear(dt)
    if yr <= 1752:
        return yr % 4 == 0
    else:
        return (yr % 4 == 0) and (yr % 100 !=0 or yr % 400 == 0)

def yeardays(dt):
    if leapyear(dt):
        return DAYS_IN_LEAP_YEAR
    else:
        return DAYS_IN_NON_LEAP_YEAR

_days_in_month =  [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
_days_in_month_so_far = [0, 0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334]


def eom(year, month):
    if month == 2 and leapyear(year):
        d = 29
    else:
        d = _days_in_month[month]
    return datetime.datetime(year, month, d)

def iseom(dt):
    """ Check if date is end of the month
    """
    dt = asdatetime(dt)
    return eom(dt.year, dt.month) == dt

现在来计算一个七天回购的远期利率，它的

- **起始日**是 **2021-10-19**
- **终止日**是 **2019-10-26**

In [68]:
date_s = '2021-10-19'
date_e = '2021-10-26'
get_forward_rate( curve, date_s, date_e, date_eval )

0.027011410850658533

In [69]:
idx = 1
date_s = schedule['floating leg schedule']['index_start_dates'][idx]
date_e = schedule['floating leg schedule']['index_end_dates'][idx]
F = get_forward_rate( curve, date_s, date_e, date_eval )
F

array([0.02722236, 0.02788022, 0.02853377, 0.02894359, 0.02598013,
       0.0260777 , 0.02617527, 0.02627284, 0.02637042, 0.02646799,
       0.02656556, 0.02666313, 0.02676071])

In [70]:
df = pd.DataFrame( F*100, index=asdatetimes(date_s), columns=['value'] )
df.reset_index().iplot( x='index', y='value', mode='lines+markers', size=10, 
                        xTitle='日期', yTitle='远期利率 (%)',
                        title=f'FR007 掉期浮动端第 {idx+1} 期中的七天回购远期利率', theme='solar' )

# <font color='#dc2624' face='微软雅黑'>5. 曲线构建</font><a name='5'></a>
[<font color='black' face='微软雅黑'>回到目录</font>](#toc)
### <font color='#2b4750' face='微软雅黑'>5.1 基本概念</font><a name='5.1'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#5)

曲线是一组金融产品的市场报价（market quote）或收益率（yield）与其到期期限之间一一对应的关系。

- 前者包含着市场信息，称为平价曲线（par curve）
- 后者用于折现现金流，称为收益曲线（yield curve）

而曲线构建就是如何将平价曲线通过拔靴（bootstrap）技巧转化成收益曲线的过程。两类曲线的到期期限都是一样的，用 $T = {T_1, T_2, \dots, T_n}$ 来表示。

<img src="Par and Yield Curve.png" width=50% height=50%>

上图比较了两类曲线的特征：

- 平价曲线严格来说是一群离散的点，因为市场上不可能在连续期限上都有报价，因此我用虚线来表示平价曲线。

- 收益曲线是用于折现各个期限的现金流的，因此需要它是连续的，这时需要设定一个内插方法将离散的收益率给连起来，因此我用实线来表示收益曲线。

此外根据使用者不同喜好，收益曲线可以用折现因子（discounting factor）、零息利率（zero-couponrate）或者即时远期利率曲线（instantaneous forward rate curve）表现出来。

<img src="Curve Representation.png" width=80% height=80%>

插值包括内插（interpolation）和外插（extrapolation）。

曲线内插方法包括线性和三次条等，内插变量包括折现因子 $P$，对数折现因子 $\text{ln}P$ 和零息利率 $r$ 等。外插通常是在零息利率 $r$ 上做平外插。

In [71]:
import numpy as np
import pandas as pd
import scipy.interpolate as spi
from scipy.optimize import fsolve

### <font color='#2b4750' face='微软雅黑'>5.2 拔靴方法</font><a name='5.2'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#5)

### <font color='red' face='微软雅黑'> 本节所有代码都来自 `curve_construction.py` 文件，专门用来构建曲线。</font>

首先需要预处理从中国外汇交易中心下载的数据。

In [72]:
MARKET['CNY']['FR007'].head()

,曲线名称,时刻,价格类型,1M,3M,6M,9M,1Y,2Y,3Y,4Y,5Y,7Y,10Y
日期,,,,,,,,,,,,,,
2021-02-10,FR007利率互换收盘曲线,16:30,均值,2.3087,2.5909,2.6475,2.6763,2.7100,2.8139,2.9068,2.9901,3.0807,3.1849,3.3374
2021-02-09,FR007利率互换收盘曲线,16:30,均值,2.6007,2.6025,2.6348,2.6613,2.6881,2.7910,2.8829,2.9653,3.0579,3.1775,3.3200
2021-02-08,FR007利率互换收盘曲线,16:30,均值,2.4899,2.6196,2.6628,2.6797,2.7062,2.8054,2.8982,2.9827,3.0756,3.1881,3.3519
2021-02-07,FR007利率互换收盘曲线,16:30,均值,2.4439,2.6100,2.6375,2.6538,2.6825,2.7888,2.8738,2.9588,3.0425,3.1738,3.3088
2021-02-05,FR007利率互换收盘曲线,16:30,均值,2.4174,2.6183,2.6550,2.6725,2.6875,2.7901,2.8838,2.9642,3.0475,3.1742,3.3232


函数 `process_data()` 做的三件事是：

1. 从数据表中获取行标签为估值日对应的哪一行数据
2. 删除无关列，如'曲线名称'、'时刻' 和 '价格类型'
3. 对每个期限生成相对应的到期日

In [73]:
def process_data( MARKET, curr, index, date_eval ):
    
    try:
        data = MARKET[curr][index][date_eval.strftime('%Y-%m-%d')]
    except:
        data = MARKET[curr][index][date_eval]
    
    data = data.drop(columns=['曲线名称', '时刻','价格类型'])
    df = pd.DataFrame( index=data.columns )
    df.index.name= 'Period'
        
    date_mat = get_maturity_date_from_trade_date( asqldate(date_eval), curr, df.index )
    
    df['Date'] = asdatetimes(date_mat)
    df['Quote'] = data.values[0]
    return df

def get_settle_date( trade_date:ql.Date, curr:str, day_lag=None ):
    day_lag = swap_day_lag(curr) if day_lag is None else day_lag
    cal = get_calendars( curr )
    settle_date = cal.advance( trade_date, ql.Period(day_lag, ql.Days) )
    return settle_date

def get_maturity_date_from_trade_date( trade_date:ql.Date, curr:str, tickers:list, day_lag=None ):
    settle_date = get_settle_date( trade_date, curr, day_lag )
    cal = get_calendars( curr )
    return [cal.advance(settle_date, ql.Period(ticker)) for ticker in tickers]

In [74]:
process_data( MARKET, 'CNY', 'FR007', date_eval )

,Date,Quote
Period,,
1M,2021-03-18,2.3087
3M,2021-05-18,2.5909
6M,2021-08-18,2.6475
9M,2021-11-18,2.6763
1Y,2022-02-18,2.7100
2Y,2023-02-20,2.8139
3Y,2024-02-19,2.9068
4Y,2025-02-18,2.9901
5Y,2026-02-18,3.0807


FR007 市场报价是一组到期日从一个月 (1M) 到十年 (10Y) 的掉期产品，它们的固定端和浮动端的付息频率都是 3 个月，注意第一个掉期到期日只有一个月，并没有完整的一期 (3 个月)。对于某一个单位本金的 FR007 掉期，它的现值可写成

    NPV = PV(固定端) - PV(浮动端)
    
对于第 $n$ 期 $[T_{n-1}, T_{n}]$，其现值公式为

<br/>
<font color='blue'>
\begin{align}
V_n(t) = P(t, T_{n}) \cdot \tau_n \cdot \left[ K - R(T_{n-1},T_n)\right]
\end{align}
</font>

<img src="7D Rate Detail.png" width=65% height=65%>

其中浮动利率 $R(T_{n-1},T_n)$ 可近似成

<br/>
<font color='blue'>
\begin{align}
R(T_{n-1},T_n) = \frac{1}{\tau_n}\left(\prod_{k=1}^{n_k}\left(1+\delta_{n,k-1}r_{n,k} \right) - 1\right)
    & = \frac{1}{\tau_n}\left(\prod_{k=1}^{n_k}\frac{P(t,s_{n,k-1})}{P(t,s_{n,k})} - 1\right) \\
    & \approx \frac{1}{\tau_n}\left(\prod_{k=1}^{n_k}\frac{P(t,t_{n,k-1})}{P(t,t_{n,k})} - 1\right)
    = \frac{1}{\tau_n}\left(\frac{P(t,T_{n-1})}{P(t,T_{n})} - 1\right)
\end{align}
</font>

带入第 $n$ 期的现值表达式可得到

<br/>
<font color='blue'>
\begin{align}
V_n(t) = P(t,T_{n}) \cdot \tau_n \cdot \left[ K - R(T_{n-1},T_n)\right] 
    &= P(t,T_{n}) \cdot \tau_n \cdot \left[ K - \frac{1}{\tau_n}\left(\frac{P(t,T_{n-1})}{P(t,T_{n})} - 1\right)\right] \\
    &= \left[ P(t,T_{n}) \cdot \tau_n \cdot K - \left(P(t,T_{n-1}) - P(t,T_{n})\right) \right]
\end{align}
</font>

将所有期的现值加总得到 FR007 掉期的 NPV，公式如下：

<br/>
<font color='blue'>
\begin{align}
V_{\text{FR007 IRS}}(t) = \sum_{n=1}^N V_n(t) 
    &= \sum_{n=1}^N \left[ P(t,T_{n}) \cdot \tau_n \cdot K - \left(P(t,T_{n-1}) - P(t,T_{n})\right) \right] \\
    &= \sum_{n=1}^N P(t,T_{n}) \cdot \tau_n \cdot K - \sum_{n=1}^N \left(P(t,T_{n-1}) - P(t,T_{n})\right) \\
    &= \underset{\substack{\downarrow \\ \text{固定端}}}{\sum_{n=1}^N P(t,T_{n}) \cdot \tau_n \cdot K} - \underset{\substack{\downarrow \\ \text{浮动端}}}{\left(P(t,T_0) - P(t,T_N)\right)}
\end{align}
</font>

### <font color='black' face='微软雅黑'>注意在定价时采用的精确表达式，在拔靴时采用的近似表达式。</font>


In [75]:
def bootstrapping( MARKET, curr, index, date_eval ):
    # 预处理从中国外汇交易中心下载的数据
    market_quote = process_data( MARKET, curr, index, date_eval )
    
    # 曲线上的标准年限：(到期日 - 估值日) / 365
    date_mat = [ asqldate(d) for d in market_quote['Date'] ]
    std_T = np.array( [ yearfrac(date_eval, d, 'Act/365') for d in date_mat ] )
    
    (fix_freq, pay_cal) = ('Quarterly', 'CNBJ')

    date_settle = get_settle_date( asqldate(date_eval), curr )
        
    r_vec = np.zeros(std_T.shape)

    for i, (ticker, inst) in enumerate(market_quote.iterrows()):

        # 1. 根据起息日、到日期、支付频率和一些关系生成日期表
        i_schedule = date_series( date_settle, date_mat[i], fix_freq, curr )

        # 2. 计算日期表到估值日的年限，用惯例 Act/365，用于插值
        i_T = np.array( [ yearfrac(date_eval, d, 'Act/365') for d in i_schedule['all_dates'] ] )

        # 3. 计算每期之间的年限，根据 CNY IRS 固定端决定，日期计数惯例 Act/365
        i_tau = np.array( [ yearfrac(ds, de, 'Act/365') for ds, de 
                            in zip(i_schedule['start_dates'], i_schedule['end_dates']) ] )

        # 4. Bootstrap 出零息利率
        K = inst['Quote'] / 100                                                 # 固定端利率
        obj_func = lambda x: compute_NPV( i, r_vec, std_T, K, i_tau, i_T, x )
        r0 = K if i==0 else r_vec[i-1]

        r = fsolve( obj_func, r0 )
        r_vec[i] = r
    
    curve = market_quote.copy()
    curve['Rate'] = r_vec*100
    curve['Discount'] = np.exp(-r_vec*std_T)
    curve = curve.set_index('Date')
    return curve


def compute_NPV( i, std_r, std_T, K, i_tau, i_T, r ):
    # 最后一个参数 r 是未知量，需要求解
    
    if i == 0: # 第一个标准日期的零息利率需要平外插 (flat extrapolation)
        
        TT = np.array( [0, std_T[i]] )
        rr = np.array( [r, r] )
        
    else: # 之后的零息利率线性内插 (linear extrapolation)
     
        TT = np.r_[0, std_T[:i], std_T[i]]        
        rr = np.r_[std_r[0], std_r[:i], r]
    
    
    # 创建样条对象 (spline object)   
    spline = spi.make_interp_spline( TT, rr, k=1 )
        
    i_r = np.squeeze( spline(i_T) )
    i_DF = np.exp(-i_r*i_T)
    
    fix_leg_PV = K * np.sum(i_tau*i_DF[1:])  
    floating_leg_PV = i_DF[0] - i_DF[-1]
    
    NPV = fix_leg_PV - floating_leg_PV
    
    return NPV

运行函数 `bootstrapping()` 拔靴出零息曲线并存储在 `Rate` 列下。

In [76]:
curr, market_index, index = 'CNY', 'FR007', 'FR 7D'
curve = bootstrapping( MARKET, curr, market_index, date_eval )
MARKET[curr][index] = curve
curve

,Quote,Rate,Discount
Date,,,
2021-03-18,2.3087,2.306658,0.997728
2021-05-18,2.5909,2.559980,0.993220
2021-08-18,2.6475,2.624956,0.986500
2021-11-18,2.6763,2.657438,0.979749
2022-02-18,2.7100,2.692966,0.972855
2023-02-20,2.8139,2.800621,0.944802
2024-02-19,2.9068,2.896246,0.916136
2025-02-18,2.9901,2.982516,0.886961
2026-02-18,3.0807,3.077392,0.856806


In [77]:
curve.reset_index().iplot( x='Date', y=['Quote','Rate'], secondary_y='Discount', mode='lines+markers', size=10, 
                           xTitle='日期', yTitle='利率 (%)',secondary_y_title='折现因子',
                           title='人民币 FR007 报价、零息利率和折现因子曲线 (2021 年 2 月 10 日)', theme='solar')

# <font color='#dc2624' face='微软雅黑'>6. 产品定价</font><a name='6'></a>
[<font color='black' face='微软雅黑'>回到目录</font>](#toc)

FR007 掉期本质上还是一个利率掉期，唯一不同的就是浮动端不再是挂钩于**一个结构性利率比如 IBOR**，而是挂钩于**一个复合利率比如由若干个 FR007 利率合成**，因此先构建普通 IRS 的类，然后 FR007 掉期作为它子类，可以继承父类普通 IRS 中的很多字段 (比如本金、起始日、到期日等) 或方法 (比如识别风险因子、固定端估值等)。

### <font color='#2b4750' face='微软雅黑'>6.1 普通利率掉期 </font><a name='6.1'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#6)

估值普通 IRS 还需要一些基本函数：

- 来自 **`utils.py`** 的判断支付端或接收端的函数 `PoR()`
- 来自 **`formatter.py`** 的美化日期和数据帧的函数 `date_formatter()` 和 `df_formatter()`
- 来自 **`schedule.py`** 的生成 IRS 日期表函数 `IRS_schedule()`、生成 IBOR 指标函数 `IBOR_date()`、根据具体估值日为支点“截取”产品日期表的 `trim_date()`
- 来自 **`market_variable.py`** 的根据历史日期获取定盘数据函数 `update_fixing()`

In [78]:
def PoR( leg ):
    Rec = ('r','rec','receive','receiving','receiver','b','buy','buying','buyer','loan')
    return 1 if leg.lower() in Rec else -1

In [79]:
def date_formatter( date, fmt='%Y/%m/%d' ):
    return [asdatetime(d).strftime(fmt) for d in date]


def df_formatter():
    money_fmt = "${:10,.2f}"
    pct_fmt = "{:.4f}%"
    date_fmt = lambda x: x.strftime('%Y/%m/%d') if not isinstance(x,str) else x
    num_fmt = "{:,.4f}"
    
    formatter = { "Cash Flow": money_fmt, 
                  "Discounted Cash Flow": money_fmt,
                  "Start Date": date_fmt,
                  "End Date": date_fmt,
                  "Fixing Date": date_fmt,
                  "Payment Date": date_fmt,
                  "IBOR Start Date": date_fmt,
                  "IBOR End Date": date_fmt,
                  "CMPR Start Date": date_fmt,
                  "CMPR End Date": date_fmt,
                  "IBOR": pct_fmt,
                  "Compound Rate": pct_fmt,
                  "Spread": pct_fmt,
                  "Tenor": num_fmt, 
                  "Discount Factor": num_fmt
                }
    return formatter

In [80]:
def trim_date( cutoff_date, schedule ):
    start_dates = schedule['start_dates']
    
    if cutoff_date < start_dates[0]:
        return schedule
    else:
        new_schedule = {}
        idx = sum([cutoff_date >= d for d in start_dates])
        for key, date_list in schedule.items():
            new_schedule[key] = date_list[idx-1:]
        return new_schedule

    
def IBOR_date( start_date, curr, index, fixing_calendar, bdc='MF' ):
    
    cal = get_calendars(fixing_calendar)
    day_lag = swap_day_lag( curr )
    IBOR_tenor = ql_period(index)
    
    # IBOR start date, end date and fixing date
    IBOR_start_date = start_date
    IBOR_end_date = [ cal.advance(d, IBOR_tenor) for d in IBOR_start_date ]
    IBOR_fixing_date = [ cal.advance(d, ql.Period(-day_lag, ql.Days)) for d in IBOR_start_date ]
    
    output = dict(index_fixing_dates=IBOR_fixing_date, 
                  index_start_dates=IBOR_start_date, 
                  index_end_dates=IBOR_end_date)
    return output

def IRS_schedule( date_eff, date_mat, curr, index, fix_freq, flt_freq, pay_cal, fixing_cal, 
                  fix_pay_delay=0, flt_pay_delay=0, bdc='MF', end_date_bdc='MF',direc='Backward', EOM=True, 
                  stub=None, date_first=None, date_last=None):
 
    fix_schedule = date_series(date_eff, date_mat, fix_freq, pay_cal, bdc, end_date_bdc, direc, EOM, stub, date_first, date_last)
    flt_schedule = date_series(date_eff, date_mat, flt_freq, pay_cal, bdc, end_date_bdc, direc, EOM, stub, date_first, date_last)
    
    IBOR_schedule = IBOR_date( flt_schedule['start_dates'], curr, index, fixing_cal )
    fix_pay_dates = shift_date( fix_schedule['end_dates'], pay_cal, fix_pay_delay )
    flt_pay_dates = shift_date( flt_schedule['end_dates'], pay_cal, flt_pay_delay )
    
    fix_schedule['payment_dates'] = fix_pay_dates
    flt_schedule['payment_dates'] = flt_pay_dates
    
    flt_schedule = {**flt_schedule, **IBOR_schedule}
    
    output = {'fixed leg schedule': fix_schedule, 'floating leg schedule': flt_schedule}
    return output

In [81]:
def update_fixing( R, fixing_date, date_eval, fixing ):
    date_eval = asdatetime(date_eval)
    fixing_date = asdatetimes(fixing_date)
        
    new_R = R.copy()
    
    for i, d in enumerate(fixing_date):
        if date_eval >= d:
            new_R[i] = fixing.loc[d,:] / 100
        else:
            break
    return new_R

下面是普通 IRS 面向对象编程的写法，用两个类表示：

- 类 **`IR_InterestRateSwap()`**：字段只包含产品的静态特征，方法只包含生成产品日期表，注意这些字段和方法都是**估值日**或者**估值方法**无关，因此也没有任何市场数据和定盘数据传入。


- 类 **`IR_InterestRateSwap_pricer()`**：传入参数估值日、市场数据、头寸数据 (即 IRS) 以及定盘数据。因为 IRS 是简单产品，其估值方法就是折现现金流，对于需要模型和数值解法来估值复杂产品，还可以传入模型 (比如 Black-Scholes 或 Bachelier) 和数值解法 (比如 PDE 或 MC) 等参数。

In [82]:
class IR_InterestRateSwap():
    def __init__(self, date_eff, date_mat, N, curr, discCurve, 
                 leg1, f1, dc1, r, 
                 leg2, f2, dc2, refCurve, s, 
                 payCLD, fixingCLD, BDC):
        
        self.date_eff = asqldate(date_eff)
        self.date_mat = asqldate(date_mat)
        self.N = N
        self.curr = curr
        self.discCurve = discCurve
        self.leg1 = leg1
        self.f1 = f1
        self.dc1 = dc1
        self.r = r/100
        self.leg2 = leg2
        self.f2 = f2
        self.dc2 = dc2
        self.refCurve = refCurve
        self.s = s/100
        self.payCLD = payCLD
        self.fixingCLD = fixingCLD
        self.BDC = BDC
        self.generate_schedule()
        
    def generate_schedule(self):
        self.schedule = IRS_schedule( self.date_eff, self.date_mat, self.curr, self.refCurve, self.f1, self.f2, 
                                      self.payCLD, self.fixingCLD, bdc=self.BDC )

类 **`IR_InterestRateSwap_pricer()`** 中构造函数 `__init__()` 中的参数：

- 估值日 `date_eval`：用于“截取”日期表，见函数 `__generate_date__()`
- 市场数据 `market`：根据具体产品从整套 `market` 获取相关的风险因子数据，见函数 `__risk_factor__()`
- 头寸数据 `deal`：几乎所有函数都用到头寸信息
- 定盘数据 `fixing`：用于确定历史时点的 IBOR 值，见函数 `_float_leg_pricer__()`

该类中函数 `NPV()` 计算产品的固定端、浮动端和整个产品的现值，函数 `display_fixed_leg()` 和 `display_floating_leg()` 将固定端和浮动端中的中间产出以数据帧的形式打印出来。

In [83]:
class IR_InterestRateSwap_pricer():
    
    def __init__( self, date_eval, market, deal, fixing ):
        self.date_eval = asqldate(date_eval)
        self.market = market
        self.deal = deal
        self.fixing = fixing
        self.__generate_date__()
    
    def __generate_date__(self):
        deal = self.deal
        self.date_settle = get_settle_date( self.date_eval, deal.curr )
        deal.schedule = {key:trim_date(self.date_settle, schedule) for key, schedule in deal.schedule.items() }
        self.deal = deal
    
    def __risk_factor__(self):
        market, deal, fixing = self.market, self.deal, self.fixing
        self.DISC_CURVE = market[deal.curr][curve_name(deal.discCurve)]
        self.REF_CURVE = market[deal.curr][curve_name(deal.refCurve)]
        self.FIXING = fixing[deal.curr][curve_name(deal.refCurve)]
    
    def __fixed_leg_pricer__(self):
        deal = self.deal
        schedule = deal.schedule['fixed leg schedule']
        date_start, date_end, date_pay = schedule['start_dates'], schedule['end_dates'], schedule['payment_dates']
        
        # 根据支付端或接收端得到 -1 或 1
        w = PoR(deal.leg1)
    
        # 根据日期计数惯例，计算每期的年限
        tau = np.array( [yearfrac(ds, de, deal.dc1) for ds, de in zip(date_start, date_end)] )
    
        # 计算每期现金流
        cash_flow = deal.N * tau * deal.r
    
        # 计算每期结束日上的折线因子
        DF = get_discount(self.DISC_CURVE, date_pay, self.date_eval)
    
        # 计算每期折线现金流
        disc_cash_flow = DF * cash_flow
    
        # 计算现值
        PV = w * np.sum(disc_cash_flow)
    
        # 打包所有中间产出
        self.output1 = {'Start Date': date_formatter(date_start), 'End Date': date_formatter(date_end), 
                        'Tenor': tau,  'Payment Date': date_formatter(date_pay),  'Discount Factor': DF, 
                        'Cash Flow': cash_flow, 'Discounted Cash Flow': disc_cash_flow,
                        'PorR':w, 'PV': PV}
    
    def __float_leg_pricer__(self):
        deal = self.deal
        schedule = deal.schedule['floating leg schedule']
        date_start, date_end, date_pay, IBOR_start_date, IBOR_end_date, IBOR_fixing_date = \
        schedule['start_dates'], schedule['end_dates'], schedule['payment_dates'], \
        schedule['index_start_dates'], schedule['index_end_dates'], schedule['index_fixing_dates']
        
        # 根据支付端或接收端得到 -1 或 1
        w = PoR(deal.leg2)

        # 根据日期计数惯例，计算每期的年限
        tau = np.array( [yearfrac(ds, de, deal.dc2) for ds, de in zip(date_start, date_end)] )

        # 计算每期的 IBOR
        IBOR = get_forward_rate( self.REF_CURVE, IBOR_start_date, IBOR_end_date, self.date_eval )

        # 更新定盘 IBOR
        IBOR = update_fixing( IBOR, IBOR_fixing_date, self.date_eval, self.FIXING )

        # 计算每期现金流
        cash_flow = deal.N * tau * (IBOR + deal.s)

        # 计算每期结束日上的折现因子
        DF = get_discount(self.DISC_CURVE, date_pay, self.date_eval)

        # 计算每期折线现金流
        disc_cash_flow = DF * cash_flow

        # 计算现值
        PV = w * np.sum(disc_cash_flow)

        # 打包所有中间产出
        self.output2 = {'Start Date': date_formatter(date_start), 'End Date': date_formatter(date_end), 
                        'Tenor': tau, 'IBOR': IBOR, 'Spread': deal.s, 
                        'Fixing Date': date_formatter(IBOR_fixing_date),
                        'IBOR Start Date': date_formatter(IBOR_start_date), 
                        'IBOR End Date': date_formatter(IBOR_end_date),
                        'Payment Date': date_formatter(date_pay),  'Discount Factor': DF, 
                        'Cash Flow': cash_flow, 'Discounted Cash Flow': disc_cash_flow,
                        'PorR':w, 'PV': PV}
    
    def NPV(self):
        self.__risk_factor__()
        self.__fixed_leg_pricer__()
        self.__float_leg_pricer__()
        fixed_leg_PV, floating_leg_PV = self.output1['PV'], self.output2['PV']
        NPV = fixed_leg_PV + floating_leg_PV
        output = {'Fixed Leg PV': fixed_leg_PV, 'Floating Leg PV': floating_leg_PV, 'NPV': NPV}
        return output
        
    def display_fixed_leg(self):
        formatter = df_formatter()
        FIX_LEG = self.output1
        non_display_columns = ['PorR','PV']
        df = pd.DataFrame(FIX_LEG).drop(columns=non_display_columns)
        return df.style.format({ k:formatter[k] for k in FIX_LEG.keys() if k not in non_display_columns} )\
                .background_gradient(subset=["Cash Flow", "Discounted Cash Flow"], cmap='PuBuGn')
        
    def display_floating_leg(self):
        formatter = df_formatter()
        FLT_LEG = self.output2
        non_display_columns = ['PorR','PV']
        df = pd.DataFrame(FLT_LEG).drop(columns=non_display_columns)
        df['IBOR'] = df['IBOR'].mul(100)
        return df.style.format({ k:formatter[k] for k in FLT_LEG.keys() if k not in non_display_columns} )\
                .background_gradient(subset=["IBOR", "Cash Flow", "Discounted Cash Flow"], cmap='PuBuGn')

### <font color='#2b4750' face='微软雅黑'>6.2 FR007 利率掉期</font><a name='6.2'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#6)

**FR007 利率掉期类**可以作为**普通利率掉期的子类**来创建，类签名如下：

    IR_CompoundIRS( IR_InterestRateSwap )
    
FR007 掉期中所有字段名和 IRS 字段名都一样，因此可以用 `super().__init__()` 来调用 类 **`IR_InterestRateSwap()`** 中的 `__init__()`。

在函数 `generate_schedule()` 中，调用 FR007 掉期产品相关的日期表生成函数 `CMPIRS_schedule()`，而不是 IRS 产品相关的期表生成函数 `IRS_schedule()`。

In [84]:
class IR_CompoundIRS( IR_InterestRateSwap ):
    def __init__(self, date_eff, date_mat, N, curr, discCurve, 
                 leg1, f1, dc1, r, 
                 leg2, f2, dc2, refCurve, s, 
                 payCLD, fixingCLD, BDC):
        super().__init__(date_eff, date_mat, N, curr, discCurve, 
                 leg1, f1, dc1, r, 
                 leg2, f2, dc2, refCurve, s, 
                 payCLD, fixingCLD, BDC)
    
    def generate_schedule(self):
        self.schedule = CMPIRS_schedule( self.date_eff, self.date_mat, self.curr, self.refCurve, self.f1, self.f2, 
                                         self.payCLD, self.fixingCLD, bdc=self.BDC )

**FR007 利率掉期估值类**可以作为**普通利率掉期估值的子类**来创建，类签名如下：

    IR_CompoundIRS_pricer( IR_InterestRateSwap_pricer )
    
 该类的主要方法有：
 
 - 构造函数 `__init__()`：继承了父类
 - 截取日期表函数 `__generate_date__()`：继承了父类
 - 获取风险因子函数 `__risk_factor__()`：继承了父类
 - 固定端估值函数 `__fixed_leg_pricer__()`：继承了父类
 - 浮动端估值函数 `__floating_leg_pricer__()`：**<font color='red'>重新编写</font>**


用户可以使用的方法包括：
 
 - 净现值函数 `NPV()`：继承了父类
 - 打印固定端中间产出函数 `display_fixed_leg()`：继承了父类
 - 打印浮动端中间产出函数 `display_floating_leg()`：**<font color='red'>重新编写</font>**
 - 可视化结果函数 `plot()`：**<font color='red'>重新编写</font>**
 

可视化的工具使用的是交互性的 **`pyecharts`**，它的特点是功能强大和制图美观。

In [85]:
from pyecharts.charts import Bar, Tab
from pyecharts.components import Table

from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
from pyecharts.globals import ThemeType

In [86]:
class IR_CompoundIRS_pricer( IR_InterestRateSwap_pricer ):
        
    def __init__( self, date_eval, market, deal, fixing ):
        super().__init__(date_eval, market, deal, fixing)
        
    def __float_leg_pricer__(self):
        deal = self.deal
        schedule = deal.schedule['floating leg schedule']
        date_start, date_end, date_pay, period_start_date, period_end_date, \
        index_start_date, index_end_date, index_fixing_date = \
        schedule['start_dates'], schedule['end_dates'], schedule['payment_dates'], \
        schedule['period_start_dates'], schedule['period_end_dates'], \
        schedule['index_start_dates'], schedule['index_end_dates'], schedule['index_fixing_dates']
                
        # 根据支付端或接收端得到 -1 或 1
        w = PoR(deal.leg2)
        
        # 根据日期计数惯例，计算每期的年限
        tau = np.array( [yearfrac(ds, de, deal.dc2) for ds, de in zip(date_start, date_end)] )

        # 计算每期的复合利率 CR
        FR007 = []
        CR = []
        CR_tau = []
        
        for i, (cmp_start, cmp_end, CR_start, CR_end, CR_fixing) in enumerate(zip(period_start_date,
                                                                                  period_end_date,
                                                                                  index_start_date,
                                                                                  index_end_date,
                                                                                  index_fixing_date)):
            
            R = get_forward_rate( self.REF_CURVE, CR_start, CR_end, self.date_eval, deal.dc2 )
                        
            if i == 0: # 更新定盘
                R = update_fixing( R, CR_fixing, self.date_eval, self.FIXING )
            
            cmp_tau = np.array( [yearfrac(ds, de, deal.dc2) for ds, de in zip(cmp_start, cmp_end)] )
            
            FR007.append(R)
            CR_tau.append(cmp_tau)
            CR.append( np.prod(1+R*cmp_tau) - 1 )

        CR = np.array(CR)/tau
        
        # 计算每期现金流
        cash_flow = deal.N * tau * (CR + deal.s)

        # 计算每期结束日上的折线因子
        DF = get_discount( self.DISC_CURVE, date_pay, self.date_eval )

        # 计算每期折线现金流
        disc_cash_flow = DF * cash_flow

        # 计算现值
        PV = w * np.sum(disc_cash_flow)

        # 打包所有中间产出
        self.output2 = {'Start Date': date_formatter(date_start), 'End Date': date_formatter(date_end), 
                        'Tenor': tau, 'Compound Rate': CR, 'Spread': deal.s, 
                        'Period Start Date': [date_formatter(d) for d in period_start_date],
                        'Period End Date': [date_formatter(d) for d in period_end_date],
                        'CR Fixing Date': [date_formatter(d) for d in index_fixing_date],
                        'CR Start Date': [date_formatter(d) for d in index_start_date], 
                        'CR End Date': [date_formatter(d) for d in index_end_date],
                        'FR007': FR007,
                        'FR007 Tenor': CR_tau,
                        'Payment Date': date_formatter(date_pay),  'Discount Factor': DF, 
                        'Cash Flow': cash_flow, 'Discounted Cash Flow': disc_cash_flow,
                        'PorR':w, 'PV': PV}


    def display_floating_leg(self):
        formatter = df_formatter()
        FLT_LEG = self.output2
        non_display_columns = ['PorR','PV','Period Start Date','Period End Date',\
                               'CR Fixing Date','CR Start Date','CR End Date','FR007','FR007 Tenor']
        df = pd.DataFrame(FLT_LEG).drop(columns=non_display_columns)
        
        df['Compound Rate'] = df['Compound Rate'].mul(100)
        return df.style.format({ k:formatter[k] for k in FLT_LEG.keys() if k not in non_display_columns} )\
                .background_gradient(subset=["Compound Rate", "Cash Flow", "Discounted Cash Flow"], cmap='PuBuGn')
    
    
    def __Table_fix__( self, data ) -> Table:
        df = pd.DataFrame(data).drop(columns=['PorR','PV'])

        table = Table()

        headers = df.columns.tolist()
        rows = df.values.tolist()

        table.add( headers, rows )
        table.set_global_opts( title_opts = opts.ComponentTitleOpts(title= f"现值 = {data['PV']}"))

        return table
  

    def __Table_flt__( self, data ):
        non_display_columns = ['PorR','PV','Period Start Date','Period End Date',\
                               'CR Fixing Date','CR Start Date','CR End Date','FR007','FR007 Tenor']
        df = pd.DataFrame(data).drop(columns=non_display_columns)
        df[['Compound Rate', 'Spread']] = df[['Compound Rate', 'Spread']].mul(100)
                
        table = Table()
        headers, rows = df.columns.tolist(), df.values.tolist()
        table.add( headers, rows )
        table.set_global_opts( title_opts = opts.ComponentTitleOpts(title= f"现值 = {data['PV']}"))
        
        df1 = pd.DataFrame(data)[['Period Start Date','Period End Date',\
                                  'CR Fixing Date','CR Start Date','CR End Date','FR007','FR007 Tenor']]
        table_list = []
        
        for i, s in df1.iterrows():
            d = {key:val for key, val in zip(s.index, s.values)}
            df_i = pd.DataFrame(d)
            df_i.index = np.arange(1,len(df_i)+1)
            df_i['FR007'] = df_i['FR007'].mul(100)
            headers, rows = df_i.columns.tolist(), df_i.values.tolist()
            table_i = Table()
            table_i.add( headers, rows )
            table_i.set_global_opts( title_opts = opts.ComponentTitleOpts(
                                                    title= f"第 {i+1} 期的复合利率 (%) = {data['Compound Rate'][i]*100}"))
            table_list.append( table_i )
        
        return table, table_list
    
    def __Bar__( self, data1, data2 ) -> Bar:

        CF1 = data1['PorR']*data1['Cash Flow']
        CF2 = data2['PorR']*data2['Cash Flow']

        label_fmt = "function(params) {return params.data.toFixed(2)}"
        tooltip_fmt = "function(params) {return '日期：' + params.name + '<br>' + '数额：' + params.data.toFixed(2)}"

        bar = ( Bar(init_opts=opts.InitOpts(theme=ThemeType.DARK))
               .add_xaxis(data1['Payment Date'])
               .add_yaxis("固定端", CF1.tolist(), category_gap="80%", label_opts=opts.LabelOpts(is_show=False))
               .add_xaxis(data2['Payment Date'])
               .add_yaxis("浮动端", CF2.tolist(), category_gap="80%", label_opts=opts.LabelOpts(is_show=False))
               .set_series_opts( label_opts=opts.LabelOpts( position="right", formatter=JsCode(label_fmt)) )
               .set_series_opts( tooltip_opts=opts.TooltipOpts( formatter=JsCode(tooltip_fmt)) )
               .set_global_opts(title_opts=opts.TitleOpts(title="现金流"),
                                xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=45, font_size=10)),
                                yaxis_opts=opts.AxisOpts(name=self.deal.curr),
                                datazoom_opts=[opts.DataZoomOpts(type_='inside')])
              )

        return bar
    
    def plot(self):
        FIX_LEG, FLT_LEG = self.output1, self.output2
        tab = Tab()
        tab.add( self.__Bar__( FIX_LEG, FLT_LEG ), "现金流图")
        tab.add( self.__Table_fix__(FIX_LEG), "固定端")
        (table, table_list) = self.__Table_flt__(FLT_LEG)
        tab.add( table, "浮动端")
        for i, t in enumerate(table_list,start=1):
            tab.add( t, f"第 {i} 期")
        return tab

### <font color='black' face='微软雅黑'>输入产品 ID</font>

In [87]:
print('输入 FR007 掉期产品的 ID: (CMPIRS1001, CMPIRS1002)')
ID = input()

输入 FR007 掉期产品的 ID: (CMPIRS1001, CMPIRS1002)
CMPIRS1001


In [88]:
deal = INSTRUMENT['IR Compound IRS'][ID]

(date_eff, date_mat, N, curr, discCurve, leg1, f1, dc1, r, leg2, f2, dc2, refCurve, s, payCLD, fixingCLD, BDC) = tuple(deal.values)
(N, r, s) = (float(N), float(r.strip('%')),  float(s.strip('%')))

cmpirs = IR_CompoundIRS( date_eff, date_mat, N, curr, discCurve, 
                         leg1, f1, dc1, r, 
                         leg2, f2, dc2, refCurve, s, 
                         payCLD, fixingCLD, BDC )

In [89]:
cmpirs_engine = IR_CompoundIRS_pricer( date_eval, MARKET, cmpirs, FIXING )
cmpirs_engine.NPV()

{'Fixed Leg PV': -4663004.760548228,
 'Floating Leg PV': 5426387.494526284,
 'NPV': 763382.7339780554}

In [90]:
cmpirs_engine.display_fixed_leg()

,Start Date,End Date,Tenor,Payment Date,Discount Factor,Cash Flow,Discounted Cash Flow
0,2021/01/20,2021/04/20,0.2466,2021/04/20,0.9954,"$591,780.82","$589,053.34"
1,2021/04/20,2021/07/20,0.2493,2021/07/20,0.9886,"$598,356.16","$591,563.66"
2,2021/07/20,2021/10/20,0.2521,2021/10/20,0.9819,"$604,931.51","$593,975.86"
3,2021/10/20,2022/01/20,0.2521,2022/01/20,0.9750,"$604,931.51","$589,833.62"
4,2022/01/20,2022/04/20,0.2466,2022/04/20,0.9683,"$591,780.82","$573,009.96"
5,2022/04/20,2022/07/20,0.2493,2022/07/20,0.9614,"$598,356.16","$575,253.26"
6,2022/07/20,2022/10/20,0.2521,2022/10/20,0.9543,"$604,931.51","$577,312.17"
7,2022/10/20,2023/01/20,0.2521,2023/01/20,0.9472,"$604,931.51","$573,002.89"


In [91]:
cmpirs_engine.display_floating_leg()

,Start Date,End Date,Tenor,Compound Rate,Spread,Payment Date,Discount Factor,Cash Flow,Discounted Cash Flow
0,2021/01/20,2021/04/20,0.2466,2.4797%,0.0000%,2021/04/20,0.9954,"$611,443.07","$608,624.96"
1,2021/04/20,2021/07/20,0.2493,2.7376%,0.0000%,2021/07/20,0.9886,"$682,528.24","$674,780.21"
2,2021/07/20,2021/10/20,0.2521,2.7312%,0.0000%,2021/10/20,0.9819,"$688,423.39","$675,955.66"
3,2021/10/20,2022/01/20,0.2521,2.7862%,0.0000%,2022/01/20,0.9750,"$702,280.95","$684,753.42"
4,2022/01/20,2022/04/20,0.2466,2.8319%,0.0000%,2022/04/20,0.9683,"$698,283.35","$676,134.30"
5,2022/04/20,2022/07/20,0.2493,2.8751%,0.0000%,2022/07/20,0.9614,"$716,810.25","$689,133.76"
6,2022/07/20,2022/10/20,0.2521,2.9293%,0.0000%,2022/10/20,0.9543,"$738,352.10","$704,641.18"
7,2022/10/20,2023/01/20,0.2521,2.9837%,0.0000%,2023/01/20,0.9472,"$752,058.02","$712,363.99"


In [92]:
cmpirs_engine.plot().render_notebook()

---
# 总结

能从头到尾学下来不容易，知识点太多，函数太多，为了之后能重复使用基本函数不断扩展代码，最后将本节所有代码结构化：
    
### 数据处理：

    |--- data_loader.py
    |    |--- load_data()
    |    |--- read_instrument()
    |    |--- read_market()
    |    |--- read_fixing()

### 日期生成：

    |--- schedule.py
    |    |--- get_settle_date()
    |    |--- get_maturity_date_from_trade_date()
    |    |--- trim_date()
    |    |--- date_series()
    |    |--- IBOR_date()
    |    |--- CMPR_date()
    |    |--- IRS_schedule()
    |    |--- CMPIRS_schedule()

### 变量计算：

    |--- market_variable.py
    |    |--- get_discount()
    |    |--- get_forward_rate()
    |    |--- update_fixing()
    
### 曲线构建：    

    |--- curve_construction.py
    |    |--- bootstrapping()
    |    |--- compute_npv()
    |    |--- process_data()

### 产品定价：

    |--- IR_CMPIRS_engine.py
    |    |--- class IR_CMPIR( IR_InterestRateSwap )
    |    |--- class IR_CMPIR_pricer( IR_InterestRateSwap_pricer )
    |
    |--- IR_InterestRateSwap_engine.py
    |    |--- class IR_InterestRateSwap()
    |    |--- class IR_InterestRateSwap_pricer()

### 其他有用的函数

    |--- utils.py               基本效用函数
    |--- ql_utils.py            和 QuantLib 有关的效用函数
    |--- date_utils.py          用于日期转换
    |--- daycount_utils.py      用于计算年限
    |--- calendar_utils.py      用于构建日历
    |--- convention_utils.py    用于获取惯例
    |--- formatter.py           用于美化格式